In [1]:
import pandas as pd
import numpy as np
import re
import plotly.express as px
from sqlalchemy import create_engine
import gspread
from gspread_dataframe import get_as_dataframe
from google.oauth2.service_account import Credentials

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### IMPOR RAW DATA

In [3]:
### FROM DATABASE UTAMA ###
# Ganti parameter berikut sesuai data kamu
host = "194.59.164.11"          # contoh: "localhost" atau "sql.yourdomain.com"
port = 3306                 # default MySQL port
user = "u870584084_0aQNr"
password = "0gWy6vkO5y"
database = "u870584084_d9QqK"

# Buat koneksi
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# Query kamu (hapus USE, karena database sudah disebut dalam connection string)
query_donate = """
SELECT    
    donasi.*,
	campaign.title program
FROM wp_dja_donate AS donasi
LEFT JOIN wp_dja_campaign AS campaign ON donasi.campaign_id = campaign.campaign_id;
"""
query_affcode = """ SELECT * FROM wp_dja_aff_code; """
query_affsubmit = """ SELECT * FROM wp_dja_aff_submit; """
query_user = """ SELECT * FROM wp_users;"""

# Eksekusi dan ambil ke DataFrame
donate = pd.read_sql(query_donate, con=engine)
affcode = pd.read_sql(query_affcode, con=engine)
affsubmit = pd.read_sql(query_affsubmit, con=engine)
user = pd.read_sql(query_user, con=engine)

In [4]:
### Join Tabel ###
donasi_submit = donate.merge(affsubmit, left_on='id', right_on='donate_id', how='left')
donasi_submit_code = donasi_submit.merge(affcode, left_on='affcode_id', right_on='id', how='left')
donasi_users = donasi_submit_code.merge(user, left_on='user_id_y', right_on='ID', how='left')

In [5]:
all_donations = donasi_users[['invoice_id', 
                              'name', 
                              'whatsapp', 
                              'main_donate', 
                              'unique_number', 
                              'nominal', 
                              'program', 
                              'payment_method', 
                              'payment_number', 
                              'payment_account', 
                              'status', 
                              'display_name', 
                              'created_at_x']]

all_donations.rename(columns={'invoice_id' : 'Invoice ID',
                      'created_at_x' : 'Date',
                      'status' : 'Payment Status',
                      'name' : 'Donatur', 
                      'whatsapp' : 'Whatsapp',
                      'main_donate' : 'Nominal',
                      'unique_number' : 'Kode Unik',
                      'nominal' : 'Total',
                      'program' : 'Program',
                      'payment_method' : 'Payment Method',
                      'payment_number' : 'Payment Number',
                      'payment_account' : 'Payment Account',
                      'display_name' : 'Fundraiser Name'
                      }, 
                    inplace=True)

# Ubah angka menjadi label
all_donations['Payment Status'] = all_donations['Payment Status'].map({1: 'Success', 0: 'Waiting'})

C:\Users\HP\AppData\Local\Temp\ipykernel_4764\351950288.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_donations.rename(columns={'invoice_id' : 'Invoice ID',
C:\Users\HP\AppData\Local\Temp\ipykernel_4764\351950288.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_donations['Payment Status'] = all_donations['Payment Status'].map({1: 'Success', 0: 'Waiting'})


In [6]:
# Pastikan kolom Tanggal sudah datetime
all_donations['Date'] = pd.to_datetime(all_donations['Date'], errors='coerce')

# Tentukan range tanggal
start_date = "2025-06-01"
end_date   = "2025-11-01"

# Filter data sesuai range
data3 = all_donations[
    (all_donations['Date'] >= start_date) 
    # & (all_donations['Date'] < end_date)
]

C:\Users\HP\AppData\Local\Temp\ipykernel_4764\915860322.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_donations['Date'] = pd.to_datetime(all_donations['Date'], errors='coerce')


In [7]:
data3['Whatsapp'] = pd.to_numeric(data3['Whatsapp'], errors='coerce')
data3 = data3.dropna(subset=['Whatsapp'])

data3['Whatsapp'] = data3['Whatsapp'].astype(int)
# data3.info()

C:\Users\HP\AppData\Local\Temp\ipykernel_4764\895373121.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['Whatsapp'] = pd.to_numeric(data3['Whatsapp'], errors='coerce')


In [8]:
# data3.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/jun-25 agus.xlsx', index=False)

In [9]:
## Data Transaksi Website Taman Zakat
# data1 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Raw Data 2025.xlsx')
# data2 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/1 juni - 30 juni.xlsx')
data1 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/Jan - April.xlsx', sheet_name = 'Donasi')
data2 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/Mei TZ.xlsx')
#data4 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/1 juli - 25 juli.xlsx')

In [10]:
## Database PIC Donatur
# database_da = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/database_da.xlsx')

### Jika terdapat perubahan database PIC
da_9009 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs Renita.xlsx')
da_8004 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs Widya.xlsx')
da_2029 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs Keken.xlsx')
da_5009 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs Udin.xlsx')
da_2032 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs Karin.xlsx')
da_2035 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs Samsul.xlsx')
da_2039 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs Dian.xlsx')
da_2037 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 2037.xlsx')
# da_2038 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 2038.xlsx')
da_2040 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 2040.xlsx')
da_2041 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 2041.xlsx')
da_2042 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 2042.xlsx')
da_2043 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 2043.xlsx')
da_2044 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 2044.xlsx')
da_2046 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 2046.xlsx')
da_2047 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 2047.xlsx')
da_2048 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 2048.xlsx')
da_2049 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 2049.xlsx')
da_2052 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 2052.xlsx')
da_2053 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 2053.xlsx')
da_2054 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 2054.xlsx')

suntikan_1 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/suntikan 1.xlsx')
suntikan_2 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/suntikan 2.xlsx')
suntikan_3 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/new 2037.xlsx')

suntikan_4 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/new 2035.xlsx')
suntikan_5 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/new 2039.xlsx')

zisco = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data zisco.xlsx')

da_2050 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 2050.xlsx')
da_2055 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 2055.xlsx')
da_2065 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 2065.xlsx')
da_9951 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 9951.xlsx')
da_9952 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 9952.xlsx')
da_9953 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 9953.xlsx')
da_9954 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 9954.xlsx')
# da_9955 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 9955.xlsx')
da_9956 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/data cs 9956.xlsx')

refil_2053 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill1 2053.xlsx')
refil_2054 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill1 2054.xlsx')

refill_2040 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 2040.xlsx')
refill_2041 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 2041.xlsx')
refill_2042 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 2042.xlsx')
refill_2043 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 2043.xlsx')
refill_2044 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 2044.xlsx')
refill_2046 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 2046.xlsx')
refill_2047 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 2047.xlsx')
refill_2048 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 2048.xlsx')
refill_2049 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 2049.xlsx')
refill_2050 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 2050.xlsx')
refill_2052 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 2052.xlsx')
refill_2055 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 2055.xlsx')
refill_2065 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 2065.xlsx')
refill_9951 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 9951.xlsx')
refill_9952 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 9952.xlsx')
refill_9953 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 9953.xlsx')
refill_9954 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 9954.xlsx')
refill_9956 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill 9956.xlsx')

refill2_2037 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 2037.xlsx')
refill2_2040 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 2040.xlsx')
refill2_2041 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 2041.xlsx')
refill2_2042 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 2042.xlsx')
refill2_2043 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 2043.xlsx')
refill2_2044 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 2044.xlsx')
# refill2_2046 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 2046.xlsx')
# refill2_2047 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 2047.xlsx')
refill2_2048 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 2048.xlsx')
refill2_2049 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 2049.xlsx')
refill2_2050 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 2050.xlsx')
refill2_2053 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 2053.xlsx')
refill2_2054 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 2054.xlsx')
# refill2_2055 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 2055.xlsx')
# refill2_2065 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 2065.xlsx')
refill2_9951 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 9951.xlsx')
# refill2_9952 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 9952.xlsx')
# refill2_9953 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 9953.xlsx')
# refill2_9954 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 9954.xlsx')
# refill2_9956 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CS/refill2 9956.xlsx')

organik = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Donatur Organik.xlsx')


# Jika semua data tersimpan dalam list:
da = [  da_9009, da_8004, da_2029, da_5009, da_2032, da_2035, da_2039, da_2037, da_2040, da_2041, da_2042, da_2043, da_2044
      , da_2046, da_2047, da_2048, da_2049, da_2052, da_2053, da_2054
      , suntikan_1, suntikan_2, suntikan_3, suntikan_4, suntikan_5
      , zisco
      , da_2050, da_2055, da_2065, da_9951, da_9952, da_9953, da_9954, da_9956
      , refil_2053, refil_2054, refill_2040, refill_2041, refill_2042, refill_2043, refill_2044, refill_2046, refill_2047, refill_2048, refill_2049, refill_2050, refill_2052, refill_2055, refill_2065
      , refill_9951, refill_9952, refill_9953, refill_9954, refill_9956
      , refill2_2037, refill2_2040, refill2_2041, refill2_2042, refill2_2043, refill2_2044
      # , refill2_2046, refill2_2047
      , refill2_2048, refill2_2049, refill2_2050, refill2_2053, refill2_2054, refill2_9951
      # , refill2_9952, refill2_9953, refill2_9954, refill2_9956
      , organik
      ]
database_da = pd.concat(da, ignore_index=True)

# Expor database_da ke Excel
database_da.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/database_da.xlsx', index=False)

In [11]:
database_da.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78267 entries, 0 to 78266
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   CRM                         78267 non-null  object        
 1   Donatur                     32270 non-null  object        
 2   Whatsapp                    78267 non-null  object        
 3   Asal                        6378 non-null   object        
 4   Bulan                       48573 non-null  object        
 5   Info                        1501 non-null   object        
 6   Nama                        45965 non-null  object        
 7   Program                     42960 non-null  object        
 8   Rata-rata                   42960 non-null  float64       
 9   kategori                    42960 non-null  object        
 10  Source                      37008 non-null  object        
 11  badge                       42960 non-null  object    

In [12]:
# Filter hanya baris yang WhatsApp-nya duplikat
duplikat_df = database_da[database_da['Whatsapp'].duplicated(keep=False)]

# Hitung jumlah CRM di data duplikat itu
crm_count_duplikat = duplikat_df['CRM'].value_counts()

print(crm_count_duplikat)


CRM
WA 9009          1699
WA 2032          1296
WA 2037           442
WA 2035           296
WA 5009           291
WA 2029           278
WA 2039           267
WA 8004           263
WA 2042            10
Zisco Anang         4
Zisco Rohadi        4
WA 2041             3
Zisco Jamal         3
Zisco Gunawan       3
WA 2040             2
new2                2
new 2037            2
new 2039            2
Zisco Agus          2
WA 2047             1
Zisco Ziyad         1
Name: count, dtype: int64


In [13]:
#### CEK DUPLIKAT WHATSAPP ####
# Contoh: pastikan tipe data Whatsapp string
database_da['Whatsapp'] = database_da['Whatsapp'].astype(str)

# Ambil hanya yang terduplikat
duplikat_df = database_da[database_da['Whatsapp'].duplicated(keep=False)]

# Urutkan agar konsisten
duplikat_df = duplikat_df.sort_values(['Whatsapp', 'CRM'])

# Tambahkan nomor urut CRM per WhatsApp
duplikat_df['CRM_no'] = duplikat_df.groupby('Whatsapp').cumcount() + 1

# Pivot agar jadi CRM1, CRM2, dll
wide_df = duplikat_df.pivot(index='Whatsapp', columns='CRM_no', values='CRM')

# Rename kolom CRM_no ke CRM1, CRM2, ...
wide_df.columns = [f'CRM{i}' for i in wide_df.columns]

# Reset index kalau mau jadi DataFrame biasa
wide_df = wide_df.reset_index()

# print(wide_df)

In [14]:
wide_df.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/database_da_wide.xlsx', index=False)

In [15]:
### Fungsi normalisasi nomor telepon
def normalize_phone(phone):
    phone = re.sub(r'\D', '', phone)  # hanya angka

    if phone.startswith('0'):
        phone = '62' + phone[1:]
    elif phone.startswith('62'):
        pass  # sudah benar
    elif phone.startswith('8'):
        phone = '62' + phone
    return phone

# Fungsi untuk menggabungkan DataFrame dengan penanganan nomor telepon
def safe_concat_phone(dfs):
    all_dfs = []

    for df in dfs:
        df = df.copy()

        if 'Whatsapp' not in df.columns:
            raise ValueError("Setiap DataFrame harus memiliki kolom 'Whatsapp'.")

        # konversi numerik ke integer terlebih dahulu
        df['Whatsapp'] = df['Whatsapp'].apply(
            lambda x: str(int(x)) if isinstance(x, (int, float, np.integer, np.floating)) and not pd.isna(x)
            else str(x)
        )

        # hapus karakter non-digit
        df['Whatsapp'] = df['Whatsapp'].apply(lambda x: re.sub(r'\D', '', x))

        # normalisasi format nomor
        df['Whatsapp'] = df['Whatsapp'].apply(normalize_phone)

        all_dfs.append(df)

    return pd.concat(all_dfs, ignore_index=True)


In [17]:
#### Impor Database Retensi CRM

## Sheet ID Rekap Himpunan 
sheet_id_organik = '1HdG6IhI8pPDtTljfKTh7sgleGSdWpVrVTZmJWSj29Yc'
sheet_id_renita = '1d7Bx_QmCoLPhWxi_3Gc1y0NA5Pf8hdY3SWwzWdbto-s'
sheet_id_udin = '1bzZmMaiQf9WnTTShOQGzreG9Qr-ErQuxob3Dw1guoQs'
sheet_id_widya = '14gQUUoLB7a4jzDIQ4IeqE7DyJrEpiJvXlgdgVyaypS4'
sheet_id_karin = '1w14CkTeZFUhdR6qN9-W5UBXtjmlbzXS_p1N5MFWE9Y8'
sheet_id_samsul = '1BqeESQoYydClz6xkqiqyiohVZMBTIIgX2_hASOHBswk'
sheet_id_dian = '1jNfl7eX3nznkOlqWKSg5RDV-MCqw7noSmaGtFSdR-wE'
sheet_id_ifti = '1eC7Ex7rO7mWczOPddL5LeyBMnwZTsxC39Btmegjlebk'
sheet_id_aisyah = '12XPLEHOutcmyTyX2xkQi0lrWr57oVjGUna0HLcrLN8U'
sheet_id_bila = '1DiTqSR-tGOWxCXzHcweYF7zSznrJoVMxeZgxLhs8_nw'
sheet_id_aris = '17wJT_L2I0zVVLOWKWENNiuxzbJC2NUFwsGXDGC6XTvc'
sheet_id_mahfud = '1uH8XSD0_hDMjEi1ALX0b0LSIpYphYo5aHGgSAUW6qJ8'
sheet_id_sinta = '178aV8SlEdwbJd2PBpDfHN2DldmS7M9O38VmmAo1be0Y'
sheet_id_bintang = '1sJXE9VXmvP5zZXa1NNQCo53xE4hw6RfCkXwcSJ3um1k'
sheet_id_aca = '10Ti-vzrC1TznQt9BL9j8L_-HS0cpWc8Bq9cd2tXrjf0'


SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
creds = Credentials.from_service_account_file('C:/Users/HP/OneDrive/Documents/KERJAAA/PYTHON/rekap-477506-4dda66e0fe0e.json', scopes=SCOPES)
gc = gspread.authorize(creds)

# daftar sheet_id
sheet_ids = [
    sheet_id_organik,
    sheet_id_renita,
    sheet_id_udin,
    sheet_id_widya,
    sheet_id_karin,
    sheet_id_samsul,
    sheet_id_dian,
    sheet_id_ifti,
    sheet_id_aisyah,
    sheet_id_bila,
    sheet_id_aris,
    sheet_id_mahfud,
    sheet_id_sinta,
    sheet_id_bintang,
    sheet_id_aca
]

all_data = []

for sid in sheet_ids:
    ws = gc.open_by_key(sid).sheet1   # bisa diganti sheet index lain bila perlu
    df = get_as_dataframe(ws, evaluate_formulas=True)
    df['Rekap Himpunan'] = sid  # tanda asal data
    all_data.append(df)

database_retensi = safe_concat_phone(all_data)

print(database_retensi.head())


       CRM     Tanggal                    Nama        Whatsapp     Bank  \
0  WA 9009  12/09/2025  Yusra Darwin Batu Bara   6285262177975      BSI   
1  WA 9009  12/09/2025        Andi Agus Salim     628539313722      BNI   
2  WA 9009  20/09/2025            lukman hakim   6281775277365      BRI   
3  WA 9009  18/09/2025             Hamba Allah  62821048960409  Mandiri   
4  WA 9009  27/09/2025               supriyati   6282213032911      BRI   

       Donasi       Akad                               Catatan Himpunan  \
0  10000000.0  Palestina           Tanya rekening dan program   WA 9009   
1   5000000.0  Palestina  tanya rekening dan program palestina  WA 9009   
2     50000.0  Palestina       Tanya lalu tf program palestina  WA 9009   
3    350000.0  Palestina       Tanya lalu tf program palestina  WA 9009   
4    100000.0     Qur'an                  Retensi website aksi  WA 9009   

  Status                                Rekap Himpunan Unnamed: 7 Unnamed: 8  
0   Done  1HdG6IhI8

In [18]:
# Expor database_retensi ke Excel
database_retensi.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/ALL Data Retensi.xlsx', index=False)

In [19]:
# database_apr = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/April/Database April.xlsx')
# database_mei = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Mei/Database Mei.xlsx')
# database_jun = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juni/Database Juni.xlsx')
# database_juli = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juli/Database Juli.xlsx')
# database_agustus = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Agust/Database Agustus.xlsx')
# database_organik = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Donatur Organik.xlsx')

In [20]:
# ### September
# renita_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 9009 Sep.xlsx')
# widya_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 8004 2028 2044 Sep.xlsx')
# udin_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 5009 2043 Sep.xlsx')
# karin_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 2029 2032 2046 2047 Sep.xlsx')
# samsul_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 2035 Sep.xlsx')
# dian_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 2039 Sep.xlsx')
# ifti_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 2037 2048 Sep.xlsx')
# bila_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 2041 2042 Sep.xlsx')
# aisyah_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 2040 2049 Sep.xlsx')
# sinta_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 9952 9953 Sep.xlsx')
# bintang_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 2055 2065 Sep.xlsx')
# mahfud_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 2050 9951 Sep.xlsx')
# balqis_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 9956 Sep.xlsx')

# september = [ renita_sep, widya_sep, udin_sep, karin_sep, 
#              samsul_sep, 
#              dian_sep, 
#              ifti_sep, bila_sep, aisyah_sep, sinta_sep, bintang_sep, 
#              mahfud_sep, 
#              balqis_sep ]
# database_september = safe_concat_phone(september)
# database_september.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Database September.xlsx', index=False)


# ### Oktober
# renita_okt = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Okt/Closing 9009 2052 okt.xlsx')
# widya_okt = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Okt/Closing 2028 2044 okt.xlsx')
# udin_okt = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Okt/Closing 5009 2043 okt.xlsx')
# karin_okt = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Okt/Closing 2029 2032 2046 2047 okt.xlsx')
# samsul_okt = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Okt/Closing 2035 okt.xlsx')
# dian_okt = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Okt/Closing 2039 okt.xlsx')
# ifti_okt = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Okt/Closing 2037 2048 okt.xlsx')
# bila_okt = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Okt/Closing 2041 2042 okt.xlsx')
# aisyah_okt = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Okt/Closing 2040 2049 okt.xlsx')
# sinta_okt = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Okt/Closing 9952 9953 okt.xlsx')
# bintang_okt = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Okt/Closing 2055 2065 okt.xlsx')
# mahfud_okt = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Okt/Closing 2050 9951 okt.xlsx')
# balqis_okt = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Okt/Closing 9954 9956 okt.xlsx')

# oktober = [ renita_okt, 
#            widya_okt, udin_okt, karin_okt, 
#              samsul_okt, 
#              dian_okt, 
#              ifti_okt, bila_okt, aisyah_okt, sinta_okt, bintang_okt, 
#              mahfud_okt, 
#              balqis_okt ]

# # ✅ Tambahkan blok pengecekan kolom DISINI
# for idx, df in enumerate(oktober, start=1):
#     print(f"File ke-{idx}:")
#     print([repr(col) for col in df.columns])
#     print()
    
# database_oktober = safe_concat_phone(oktober)
# database_oktober.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Okt/Database Oktober.xlsx', index=False)

# # Gabungkan data retensi
# retensi = [  
#             # all_retensi,
#             database_organik
#             , database_apr
#             , database_mei
#             , database_jun
#             , database_juli
#             , database_agustus
#             , database_september
#             , database_oktober
#            ] 
# database_retensi = safe_concat_phone(retensi)

# # Expor database_retensi ke Excel
# database_retensi.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/ALL Data Retensi.xlsx', index=False)

In [21]:
# ## Data Retensi dan CTWA sebelumnya
# # all_retensi = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/ALL Data Retensi.xlsx')

# # Data Retensi Per CS
# renita_apr = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/April/Closing Renita April.xlsx')
# widya_apr = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/April/Closing Widya April.xlsx')
# udin_apr = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/April/Closing Udin April.xlsx')

# april = [renita_apr, widya_apr, udin_apr]
# database_apr = pd.concat(april, ignore_index=True)
# database_apr.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/April/Database April.xlsx', index=False)

# renita_mei = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Mei/Closing Renita Mei.xlsx')
# widya_mei = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Mei/Closing Widya Mei.xlsx')
# keken_mei = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Mei/Closing Keken Mei.xlsx')
# udin_mei = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Mei/Closing Udin Mei.xlsx')

# mei = [renita_mei, widya_mei, keken_mei, udin_mei]
# database_mei = pd.concat(mei, ignore_index=True)

# database_mei.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Mei/Database Mei.xlsx', index=False)

# ### Juni
# renita_jun = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juni/Closing Renita Juni.xlsx')
# widya_jun = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juni/Closing Widya Juni.xlsx')
# keken_jun = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juni/Closing Keken Juni.xlsx')
# udin_jun = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juni/Closing Udin Juni.xlsx')
# karin_jun = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juni/Closing Karin Juni.xlsx')
# samsul_jun = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juni/Closing Samsul Juni.xlsx')
# dian_jun = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juni/Closing Dian Juni.xlsx')

# jun = [renita_jun, widya_jun, keken_jun, udin_jun, karin_jun, samsul_jun, dian_jun]
# database_jun = pd.concat(jun, ignore_index=True)

# database_jun.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juni/Database Juni.xlsx', index=False)

# ### Juli
# renita_jul = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juli/Closing Renita Juli.xlsx')
# widya_jul = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juli/Closing Widya Juli.xlsx')
# keken_jul = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juli/Closing Keken Juli.xlsx')
# udin_jul = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juli/Closing Udin Juli.xlsx')
# karin_jul = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juli/Closing Karin Juli.xlsx')
# samsul_jul = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juli/Closing Samsul Juli.xlsx')
# dian_jul = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juli/Closing Dian Juli.xlsx')
# ifti_jul = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juli/Closing Ifti Juli.xlsx')

# keken_jul['Whatsapp'] = keken_jul['Whatsapp'].astype(float).astype('Int64')
# samsul_jul['Whatsapp'] = samsul_jul['Whatsapp'].astype(float).astype('Int64')

# juli = [renita_jul, widya_jul, keken_jul, udin_jul, karin_jul, samsul_jul, dian_jul, ifti_jul]
# database_juli = pd.concat(juli, ignore_index=True)

# database_juli.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Juli/Database Juli.xlsx', index=False)

# ### Agustus
# renita_aug = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Agust/Closing 9009.xlsx')
# widya_aug = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Agust/Closing 8004 2028 2044.xlsx')
# keken_aug = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Agust/Closing 2029.xlsx')
# udin_aug = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Agust/Closing 5009.xlsx')
# karin_aug = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Agust/Closing 2032.xlsx')
# samsul_aug = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Agust/Closing 2035.xlsx')
# dian_aug = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Agust/Closing 2039.xlsx')
# ifti_aug = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Agust/Closing 2037.xlsx')
# bila_aug = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Agust/Closing 2041 2042.xlsx')
# aisyah_aug = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Agust/Closing 2040 2049.xlsx')

# samsul_aug['Whatsapp'] = samsul_aug['Whatsapp'].astype(float).astype('Int64')

# agustus = [
#     renita_aug, 
#     widya_aug, 
#     keken_aug, 
#     udin_aug, karin_aug
#     , samsul_aug
#     , dian_aug
#     , ifti_aug
#     , bila_aug
#     , aisyah_aug
#     ]
# database_agustus = pd.concat(agustus, ignore_index=True)
# database_agustus.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Agust/Database Agustus.xlsx', index=False)

# ### September
# renita_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 9009 Sep.xlsx')
# widya_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 8004 2028 2044 Sep.xlsx')
# udin_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 5009 2043 Sep.xlsx')
# karin_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 2029 2032 2046 2047 Sep.xlsx')
# # samsul_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 2035 Sep.xlsx')
# dian_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 2039 Sep.xlsx')
# ifti_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 2037 2048 Sep.xlsx')
# bila_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 2041 2042 Sep.xlsx')
# aisyah_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 2040 2049 Sep.xlsx')
# sinta_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 9952 9953 Sep.xlsx')
# bintang_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 2055 2065 Sep.xlsx')
# mahfud_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 2050 9951 Sep.xlsx')
# balqis_sep = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Closing 9956 Sep.xlsx')

# september = [ renita_sep, widya_sep, udin_sep, karin_sep, 
#             #  samsul_sep, 
#              dian_sep, 
#              ifti_sep, bila_sep, aisyah_sep, sinta_sep, bintang_sep, 
#              mahfud_sep, 
#              balqis_sep ]
# database_september = pd.concat(september, ignore_index=True)
# database_september.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/Sep/Database September.xlsx', index=False)

# # Gabungkan data retensi
# retensi = [  
#             # all_retensi,
#             database_apr
#             , database_mei
#             , database_jun
#             , database_juli
#             , database_agustus
#             , database_september
#            ] 
# database_retensi = pd.concat(retensi, ignore_index=True)

# # Expor database_retensi ke Excel
# database_retensi.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CLOSING CS/ALL Data Retensi.xlsx', index=False)

In [22]:
database_retensi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6794 entries, 0 to 6793
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CRM             6794 non-null   object 
 1   Tanggal         6794 non-null   object 
 2   Nama            6794 non-null   object 
 3   Whatsapp        6794 non-null   object 
 4   Bank            6758 non-null   object 
 5   Donasi          6794 non-null   float64
 6   Akad            6792 non-null   object 
 7   Catatan         12 non-null     object 
 8   Himpunan        14 non-null     object 
 9   Status          14 non-null     object 
 10  Rekap Himpunan  6794 non-null   object 
 11  Unnamed: 7      154 non-null    object 
 12  Unnamed: 8      16 non-null     object 
dtypes: float64(1), object(12)
memory usage: 690.1+ KB


In [23]:
## Data CTWA
ctwa1 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CTWA/CTWA Mei.xlsx')
ctwa2 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CTWA/CTWA Udin.xlsx')
ctwa3 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CTWA/CTWA 9009.xlsx', sheet_name='Rekap_CTWA')
ctwa4 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CTWA/CTWA 2032.xlsx', sheet_name='Rekap_CTWA')
ctwa5 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CTWA/CTWA 2035.xlsx', sheet_name='Rekap_CTWA')
ctwa6 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CTWA/CTWA 2039.xlsx', sheet_name='Rekap_CTWA')
ctwa7 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CTWA/CTWA Widya.xlsx', sheet_name='CTWA')
ctwa8 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CTWA/CTWA 2050.xlsx', sheet_name='Rekap_CTWA')
ctwa9 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CTWA/CTWA 2055.xlsx', sheet_name='Rekap_CTWA')
ctwa10 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CTWA/CTWA 2065.xlsx', sheet_name='Rekap_CTWA')
ctwa11 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CTWA/CTWA 9951.xlsx', sheet_name='Rekap_CTWA')
ctwa12 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CTWA/CTWA 9952.xlsx', sheet_name='Rekap_CTWA')
ctwa13 = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/CTWA/CTWA 9953.xlsx', sheet_name='Rekap_CTWA')


# Gabungkan semua data
ctwa = pd.concat([ctwa1, ctwa2, ctwa3, ctwa4, ctwa5, ctwa6, ctwa7, ctwa8, ctwa9, ctwa10, ctwa11, ctwa12, ctwa13], ignore_index=True)

c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\HP\AppData\Local\Programs\

In [24]:
# invalid = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/INPUT/Invalid.xlsx')
# update_prioritas = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas.xlsx") ## Update jadi Kategori Prioritas with Last Action New
# update_2029 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Daily Activity CS/Daily 2/Update Daily 2029.xlsx")
# update_bonding_2037 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas 2037.xlsx") # tdk ada last_date (data awal di /Daily 2/Daily WA 2037.xlsx)
# update_bonding_2037_tambahan = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas 2037 tambahan.xlsx") # tdk ada last_date (data awal di /Daily 2/Daily WA 2037 tambahan.xlsx)
# update_bonding_2039 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas 2035 2039.xlsx") # tdk ada last_date (data awal di /Daily 3/)
# update_bonding_2032_tambahan = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas 2032 tambahan.xlsx") # tdk ada last_date (data awal di /Daily 3/)
# update_bonding_28_40_41_42 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas 28 40 41 42.xlsx") # tdk ada last_date (data awal di /Daily 3/)  Semua Daily 3 di atas sudah digabungkan di Kategori PrioritasWA 35 39 32plus 28 40 41 42 New.xlsx
# update_bonding_53_54 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas 53 54.xlsx") # tdk ada last_date (data awal di /Daily 4/Bonding)
# update_bonding_48_49 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas 48 49.xlsx") # tdk ada last_date (data awal di /Daily 4/Bonding)
# update_5009_44 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas 5009 44.xlsx") # tdk ada last_date
# update_2032 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas 2032.xlsx") # tdk ada last_date
# update_37_41_50_9951 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas 37 41 50 9951.xlsx") # tdk ada last_date
# update_42_43 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas 42 43.xlsx") # tdk ada last_date
# update_40 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas 40.xlsx")  # tdk ada last_date  ( Kategori Prioritas WA 32 5009 42 37 9951 44 50 43 41 ref 40.xlsx)
# update_37_41 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas ref 37 41.xlsx")  # tdk ada last_date
# update_2055_2065_9952_9953 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas WA 2055 2065 9952 9953.xlsx") # Update (Kategori Prioritas WA 2055 2065 9952 9953 new37 ref41.xlsx)
# update_2047 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas WA 2047.xlsx") # tdk ada last_date

# kategori_update = pd.concat([invalid, update_prioritas, update_2029, update_bonding_2037, update_bonding_2037_tambahan, update_bonding_2039
#                              , update_bonding_2032_tambahan, update_bonding_28_40_41_42, update_bonding_53_54, update_bonding_48_49
#                              , update_5009_44, update_2032, update_37_41_50_9951, update_42_43, update_40, update_37_41, update_2055_2065_9952_9953, update_2047
#                              ], ignore_index=True)

In [25]:
invalid = pd.read_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/INPUT/Invalid.xlsx')
update_prioritas = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas with Last Action New.xlsx")
update_2029 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Daily Activity CS/Daily 2/Update Daily 2029.xlsx")
update_2037 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas WA 2037 New.xlsx")
update_daily3 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas WA 35 39 32plus 28 40 41 42 New.xlsx")
update_48_49_53_54 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas WA 48 49 53 54 New.xlsx")
update_daily4 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas WA 32 5009 42 37 9951 44 50 43 41 ref 40.xlsx")
update_bonding = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas WA 2055 2065 9952 9953 new37 ref41.xlsx")
update_2047 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas WA 2047.xlsx")
update_9009 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas WA 9009 Karantina.xlsx")
update_9954_9956_ref1_49_53_54 = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Prioritas WA 9954 9956 ref1 49 53 54.xlsx")

kategori_update = pd.concat([invalid, update_prioritas, update_2029, update_2037, update_daily3, update_48_49_53_54, update_daily4, update_bonding
                             , update_2047, update_9009, update_9954_9956_ref1_49_53_54
                             ], ignore_index=True)

In [26]:
kategori_update.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72681 entries, 0 to 72680
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Bulan               1330 non-null   float64       
 1   Donatur             66117 non-null  object        
 2   Whatsapp            72681 non-null  int64         
 3   Program             61161 non-null  object        
 4   Rata-rata           1330 non-null   float64       
 5   kategori            72679 non-null  object        
 6   Source              1330 non-null   object        
 7   badge               1330 non-null   object        
 8   Periode Blast       1330 non-null   object        
 9   Validasi            72680 non-null  object        
 10  Kategori_Prioritas  71349 non-null  object        
 11  No CS               64796 non-null  object        
 12  Donasi Sebelumnya   15219 non-null  float64       
 13  last_respon         56297 non-null  object    

In [27]:
## Rename CRM kategori update
kategori_update.loc[kategori_update['No CS'] == 'Syifa 5009', 'No CS'] = 'WA 5009'
kategori_update.loc[kategori_update['No CS'] == 'Hana 8004', 'No CS'] = 'WA 8004'
kategori_update.loc[kategori_update['No CS'] == 'Anisa 9009', 'No CS'] = 'WA 9009'
kategori_update.loc[kategori_update['No CS'] == 'Nia 2029', 'No CS'] = 'WA 2029'
kategori_update.loc[kategori_update['Validasi'] == 'Number not registered', 'Validasi'] = 'Invalid'

In [28]:
kategori_update.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Update.xlsx', index=False)

In [29]:
## Drop duplikat di kategori update
kategori_update = kategori_update.drop_duplicates(subset=['Whatsapp'], keep='last')

In [30]:
kategori_update['Whatsapp'].duplicated().sum()

np.int64(0)

In [31]:
### Database Kategori Donatur
# Kategori Berdasarkan Daily Activity CRM
# database_daily = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Kategori Donatur.xlsx")

# # Kategori Invalid
# database_invalid = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/DATABASE INVALID.xlsx")

# # Kategori Pasif
# database_pasif = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/DATABASE PASIF.xlsx")

# # menyesuaikan nama kolom jika ada nama kolom yang belum sesuai standarisasi
# kolom = {"CRM": "No CS"}
# # rename
# database_invalid = database_invalid.rename(columns=kolom)
# database_pasif = database_pasif.rename(columns=kolom)

# ### Gabungkan semua database
# kategori_update = pd.concat([database_daily, database_invalid, database_pasif], ignore_index=True)

#### Cleansing Data

In [32]:
# # 1. Jika kolom mengandung NaN, isi dulu dengan nilai sementara atau hilangkan baris NaN
# Cara 1: pakai regex di pandas
data3['Whatsapp'] = data3['Whatsapp'].astype(str).str.replace(r'\D', '', regex=True)

# data['Whatsapp'] = data['Whatsapp'].astype(int)
data1['Whatsapp'] = data1['Whatsapp'].astype(int)
data2['Whatsapp'] = data2['Whatsapp'].astype(int)
data3['Whatsapp'] = data3['Whatsapp'].astype(int)

# # 2. Gabungkan kedua DataFrame
data = pd.concat([data1, data2, data3], ignore_index=True)

## Standarisasi Tipe Data
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data['Whatsapp'] = data['Whatsapp'].astype(int)
data['Total'] = data['Total'].fillna(0).astype(int)

## Expor data ke Excel
data.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Raw Data 2025.xlsx', index=False)

In [33]:
# Filter tanggal 26 - 30 Juni 2025 (atau sesuaikan tahunnya jika perlu)
mask = (data['Date'] >= '2025-05-26') & (data['Date'] <= '2025-06-30')

# Ambil datanya
data_juni = data[mask]

len(data_juni)  # Menampilkan jumlah baris data yang diambil

21312

In [34]:
########## Bersihkan Duplikat #############
# # Bersihkan da_9009
# da_9009 = da_9009.drop_duplicates(subset='Whatsapp', keep='first')
# da_9009 = da_9009.dropna(subset=['Whatsapp'])

# # Bersihkan da_8004
# da_8004 = da_8004.drop_duplicates(subset='Whatsapp', keep='first')
# da_8004 = da_8004.dropna(subset=['Whatsapp'])

# # Bersihkan da_2029
# da_2029 = da_2029.drop_duplicates(subset='Whatsapp', keep='first')
# da_2029 = da_2029.dropna(subset=['Whatsapp'])

# # Bersihkan da_5009
# da_5009 = da_5009.drop_duplicates(subset='Whatsapp', keep='first')
# da_5009 = da_5009.dropna(subset=['Whatsapp'])

# # Bersihkan da_2032
# da_2032 = da_2032.drop_duplicates(subset='Whatsapp', keep='first')
# da_2032 = da_2032.dropna(subset=['Whatsapp'])

In [35]:
# Cleansing Data Retensi
# Pastikan kolom 'Whatsapp' hanya berisi angka, lalu kosongkan jika tidak ada angka
database_retensi['Whatsapp'] = database_retensi['Whatsapp'].astype(str).str.replace(r'\D', '', regex=True)

# Drop baris dengan Whatsapp kosong ('') atau null
database_retensi = database_retensi[database_retensi['Whatsapp'].str.strip() != '']
database_retensi = database_retensi[database_retensi['Whatsapp'].notna()]

# Konversi ke integer setelah data bersih
database_retensi['Tanggal'] = pd.to_datetime(database_retensi['Tanggal'], errors='coerce', dayfirst=True)
database_retensi['Whatsapp'] = database_retensi['Whatsapp'].astype(float).astype('Int64')
database_retensi['Bank'] = database_retensi['Bank'].astype(str).str.strip()
database_retensi['Donasi'] = database_retensi['Donasi'].fillna(0).astype(int)
database_retensi['Akad'] = database_retensi['Akad'].astype(str).str.strip()


# Tambahkan kolom 'Payment Status' di database retensi
database_retensi['Payment Status'] = 'Success'
# Tambahkan kolom 'is_random' di database retensi
database_retensi['is_random'] = True

# Simpan database retensi ke Excel
database_retensi.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/ALL Data Retensi.xlsx', index=False)

In [36]:
#database_retensi = pd.concat([database_retensi, allretensi], ignore_index=True)

In [37]:
# Pastikan kolom 'Whatsapp' hanya berisi angka, lalu kosongkan jika tidak ada angka
ctwa['Whatsapp'] = ctwa['Whatsapp'].astype(str).str.replace(r'\D', '', regex=True)

# Drop baris dengan Whatsapp kosong ('') atau null
ctwa = ctwa[ctwa['Whatsapp'].str.strip() != '']
ctwa = ctwa[ctwa['Whatsapp'].notna()]

# Konversi tipe data setelah data bersih
ctwa['Tanggal'] = pd.to_datetime(ctwa['Tanggal'], errors='coerce')
ctwa['Whatsapp'] = ctwa['Whatsapp'].astype(int)
ctwa['Donasi'] = ctwa['Donasi'].fillna(0).astype(float).astype(int)
ctwa['Akad'] = ctwa['Akad'].astype(str).str.strip()

# Tambahkan kolom tambahan
ctwa['is_random'] = True
ctwa['Source'] = 'CTWA'


#### Database Add

In [38]:
database_add = pd.concat([database_retensi, ctwa], ignore_index=True)
database_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9518 entries, 0 to 9517
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   CRM             9478 non-null   object        
 1   Tanggal         9489 non-null   datetime64[ns]
 2   Nama            9431 non-null   object        
 3   Whatsapp        9518 non-null   Int64         
 4   Bank            7243 non-null   object        
 5   Donasi          9518 non-null   int64         
 6   Akad            9518 non-null   object        
 7   Catatan         21 non-null     object        
 8   Himpunan        14 non-null     object        
 9   Status          14 non-null     object        
 10  Rekap Himpunan  6787 non-null   object        
 11  Unnamed: 7      154 non-null    object        
 12  Unnamed: 8      16 non-null     object        
 13  Payment Status  8696 non-null   object        
 14  is_random       9518 non-null   bool          
 15  Nama

In [39]:
## Standarisasi nama kolom untuk database retensi
database_add.rename(columns={'Tanggal' : 'Date',
                             'Nama' : 'Donatur', 
                             'Bank' : 'Payment Account', 
                             'Donasi' : 'Total',
                             'Akad' : 'Program'}, 
                    inplace=True)

In [40]:
# database_add.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Raw Data 2025 Retensi & CTWA.xlsx', index=False)

In [41]:
data_ads = data[['Whatsapp', 'Donatur', 'Sapaan', 'Email', 'Date', 'Total', 'Comment', 'Payment Status', 'Payment Number', 'Payment Account', 'Fundraiser Name', 'Program', 'UTM Source']]

#### Validasi & Standarisasi Nomor Whatsapp

In [42]:
#### VALIDASI NOMOR WHATSAPP ####
# Pastikan Whatsapp hanya berisi angka random
def is_random_number(num):
    num = str(num)

    # Nomor harus hanya angka
    if not num.isdigit():
        return False

    # Harus valid dari segi panjang dan awalan
    starts_valid = num.startswith("08") or num.startswith("8") or num.startswith("62")
    length_valid = 8 < len(num) <= 14

    if not (starts_valid and length_valid):
        return False

    # Cek jika terlalu banyak angka sama (seperti 08888888888)
    if len(set(num)) <= 3:
        return False

    # Cek pola berulang (seperti 081234567812345678)
    if len(num) > 6 and num[:int(len(num)/2)] == num[int(len(num)/2):]:
        return False

    # Cek pola urutan (seperti 08123456789)
    if num[2:].isdigit() and num[2:] in '1234567890'*2:
        return False

    return True

data_ads['is_random'] = data_ads['Whatsapp'].apply(is_random_number)
data_ads[data_ads['is_random'] == False].head()

C:\Users\HP\AppData\Local\Temp\ipykernel_4764\255535027.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ads['is_random'] = data_ads['Whatsapp'].apply(is_random_number)


,Whatsapp,Donatur,Sapaan,Email,Date,Total,Comment,Payment Status,Payment Number,Payment Account,Fundraiser Name,Program,UTM Source,is_random
7,178432473,Vickyadi,Bapak,NaN,2025-02-28 09:51:00,100985,NaN,Waiting,944995941501,VA CIMB_NIAGA - iPaymu Taman Zakat Indone,Taman Zakat 1,BANGUN KEMBALI GAZA,NaN,False
30,123456789,Hamba Allah,Ibu,NaN,2025-02-28 05:03:00,50980,"Semoga Allah swt mengampuni dosa, memberikan k...",Success,9881988123417464,VA BNI - iPaymu Taman Zakat Indonesia,Taman Zakat 1,SEDEKAH/INFAK HARIAN,NaN,False
73,123456789,Hamba Allah,Ibu,NaN,2025-02-27 06:52:00,50929,"Semoga Allah swt mengampuni dosa, diberi kemud...",Waiting,3811800086525823,VA BCA - Hallo Bali,Taman Zakat 1,SEDEKAH/INFAK HARIAN,NaN,False
179,9876544,Mia,Ibu,NaN,2025-02-23 06:18:00,100902,NaN,Success,Bank Mandiri 1410075075002,Taman Zakat Indonesia,Taman Zakat 1,SEDEKAH KEMANUSIAAN: BANGUN KEMBALI PALESTINA,NaN,False
221,0,W,Bapak,NaN,2025-02-21 13:30:00,100991,Doa selamat dunia dan akhirat..,Success,BRI 021101002263302,Taman Zakat Indonesia,Taman Zakat 1,SEDEKAH UNTUK PALESTINA,NaN,False


In [43]:
#### NORMASLISASI NOMOR WA
def normalisasi_wa(nomor):
    nomor = str(nomor).strip()
    if nomor.startswith('0'):
        return '62' + nomor[1:]
    elif nomor.startswith('8'):
        return '62' + nomor
    elif nomor.startswith('+62'):
        return '62' + nomor[3:]
    elif nomor.startswith('62'):
        return nomor
    else:
        return nomor  # fallback

# Terapkan ke kedua data
data_ads['WA_Norm'] = data_ads['Whatsapp'].apply(normalisasi_wa)


C:\Users\HP\AppData\Local\Temp\ipykernel_4764\672677075.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ads['WA_Norm'] = data_ads['Whatsapp'].apply(normalisasi_wa)


In [44]:
#DROP data yang tidak diperlukan
data_ads = data_ads.drop(['Whatsapp'], axis=1)
# menyesuaikan nama kolom jika ada nama kolom yang belum sesuai standarisasi
kolom = {"WA_Norm": "Whatsapp"}
# rename
data_ads = data_ads.rename(columns=kolom)
# source
data_ads['Source'] = 'Web Ads'

In [45]:
# Jumlah Whatsapp unik untuk data random dan payment SUCCESS
data_ads[(data_ads['is_random'] == True) & (data_ads['Payment Status'] == 'Success')]['Whatsapp'].nunique()


81132

In [46]:
# Jumlah Whatsapp unik untuk data random dan payment SUCCESS
data_ads[(data_ads['is_random'] == False)]['Whatsapp'].nunique()

984

#### CRM Fundraiser Name & CS

In [47]:
### Standarisasi data agar dapat di kelompokkan
# Daftar kolom yang ingin diproses
target_cols = ['Fundraiser Name']

# Proses hanya kolom tertentu
for col in target_cols:
    if col in data_ads.columns:  # cek agar tidak error jika kolom tidak ada
        data_ads[col] = (
            data_ads[col]
            .astype(str)
            .str.strip()
            .str.upper()
            .str.replace('_', ' ', regex=False)
        )

In [48]:
### Mengambil data Fundraiser Name untuk CRM
# Assign CRM based on Fundraiser Name
def assign_crm(name):
    if name == 'INDRA':
        return 'Zisco Indra'
    elif name == 'MOH JAMALUDIN':
        return 'Zisco Jamal'
    elif name == 'ANANG MASTOLKA':
        return 'Zisco Anang'
    elif name == 'ZIYAD':
        return 'Zisco Ziyad'
    elif name == 'AGUS HENDRIAWAN':
        return 'Zisco Agus'
    elif name == 'LULUK WAHYU':
        return 'Zisco Luluk'
    elif name == 'MOCH ROHADI':
        return 'Zisco Rohadi'
    elif name == 'FAHRIZAL IDHAM P.':
        return 'Zisco Idham'
    elif name == 'MUHAMMAD RIFKY NAILUL FALAQ':
        return 'Zisco Rifky'
    elif name == 'FANDI F':
        return 'Zisco Fandi'
    elif name == 'DEDI SETIAWAN':
        return 'Zisco Dedi'
    elif name == 'TAUFI QURRACHMAN':
        return 'Zisco Taufi'
    elif name == 'GUNAWAN':
        return 'Zisco Gunawan'
    # elif name in ['Citra', 'Dina']:
    #     return 'WA 2028'
    else:
        return np.nan

data_ads['CRM'] = data_ads['Fundraiser Name'].apply(assign_crm)


In [49]:
## Patikan tidak ada duplikasi di database_da
database_da['Whatsapp'].duplicated().sum()
# Menghapus duplikat berdasarkan kolom 'Whatsapp'       
database_da = database_da.drop_duplicates(subset='Whatsapp', keep='first')
# Menghapus baris yang memiliki nilai NaN pada kolom 'Whatsapp' 
database_da = database_da.dropna(subset=['Whatsapp'])

In [50]:
## 
database_da['Whatsapp'] = database_da['Whatsapp'].astype(str).str.replace(r'\D', '', regex=True)

## Pastikan kedua kolom Whatsapp dalam bentuk integer
data_ads['Whatsapp'] = data_ads['Whatsapp'].astype(int)
database_da['Whatsapp'] = database_da['Whatsapp'].astype(int)

In [51]:
# Lakukan merge dengan data CRM
data_ads = data_ads.merge(database_da[['Whatsapp', 'CRM']], on='Whatsapp', how='left', suffixes=('', '_new'))

# Isi kolom CRM yang kosong dengan data dari CRM_new (hasil join)
data_ads['CRM'] = data_ads['CRM'].fillna(data_ads['CRM_new'])

# Hapus kolom CRM_new (karena sudah dipakai)
data_ads = data_ads.drop(columns=['CRM_new'])

##### Pemetaan CRM by Periode

In [52]:
# #################### Pemetaan CRM berdasarkan rentang tanggal ######################
# # Isi CRM berdasarkan rentang tanggal
# data_ads.loc[
#     (data_ads['Date'] >= '2025-04-30') &
#     (data_ads['Date'] <= '2025-05-20') &
#     (data_ads['CRM'].isna()),
#     'CRM'
# ] = 'WA 2028'


In [53]:
data_ads['CRM'].value_counts(dropna=False)

CRM
NaN             25156
WA 9009          9335
WA 8004          6758
WA 5009          6648
WA 2029          6391
WA 2032          5865
new2             3648
new1             3517
WA 2039          1900
WA 2042          1872
WA 2035          1840
WA 2040          1820
WA 2037          1628
new 2037         1365
new 2039         1234
WA 2047          1218
WA 2053          1214
WA 2043          1208
new 2035         1178
WA 2041          1122
WA 2046          1085
WA 2052          1084
WA 2054          1073
WA 2048          1072
WA 2044          1060
WA 2049          1057
WA 9952          1044
WA 9954          1040
WA 9953          1035
WA 2065          1034
WA 9951          1034
WA 2055          1025
WA 2050          1025
WA 9956           991
refill 2044       792
refill 2046       778
refill 2043       775
refill 2047       762
refill 2041       761
refill 9951       749
new 2054          740
refill 2049       736
refill 2048       733
refill 9954       732
refill 9956       727
refill

#### Merge Data Retensi dan CTWA

In [54]:
#### Pastikan kedua kolom memiliki tipe Data yang sama
data_ads.info()
database_retensi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120824 entries, 0 to 120823
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Donatur          120823 non-null  object        
 1   Sapaan           49205 non-null   object        
 2   Email            17452 non-null   object        
 3   Date             120824 non-null  datetime64[ns]
 4   Total            120824 non-null  int64         
 5   Comment          29229 non-null   object        
 6   Payment Status   120824 non-null  object        
 7   Payment Number   120824 non-null  object        
 8   Payment Account  120824 non-null  object        
 9   Fundraiser Name  120824 non-null  object        
 10  Program          120824 non-null  object        
 11  UTM Source       17039 non-null   object        
 12  is_random        120824 non-null  bool          
 13  Whatsapp         120824 non-null  int64         
 14  Source           120

In [55]:
# Cek dan buang kolom yang sama namanya dua kali
data_ads = data_ads.loc[:, ~data_ads.columns.duplicated()]
database_add = database_add.loc[:, ~database_add.columns.duplicated()]

# Satukan (append baris-baris dari dua DataFrame)
database_all = pd.concat([data_ads, database_add], ignore_index=True)

In [56]:
database_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130342 entries, 0 to 130341
Data columns (total 30 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Donatur          130254 non-null  object        
 1   Sapaan           49205 non-null   object        
 2   Email            17452 non-null   object        
 3   Date             130313 non-null  datetime64[ns]
 4   Total            130342 non-null  int64         
 5   Comment          29229 non-null   object        
 6   Payment Status   129520 non-null  object        
 7   Payment Number   120824 non-null  object        
 8   Payment Account  128067 non-null  object        
 9   Fundraiser Name  120824 non-null  object        
 10  Program          130342 non-null  object        
 11  UTM Source       17039 non-null   object        
 12  is_random        130342 non-null  bool          
 13  Whatsapp         130342 non-null  Int64         
 14  Source           123

#### Pemetaan Kategori Program dan Payment

In [57]:
# Daftar kolom yang ingin diproses
target_cols = ['Program', 'Payment Status', 'Payment Number', 'Payment Account']

# Proses hanya kolom tertentu
for col in target_cols:
    if col in database_all.columns:  # cek agar tidak error jika kolom tidak ada
        database_all[col] = (
            database_all[col]
            .astype(str)
            .str.replace(r'\s+', ' ', regex=True)  # Hapus spasi berlebih
            .str.strip()                           # Hapus spasi di awal/akhir
            .str.upper()
            .str.replace('_', ' ', regex=False)
        )


In [58]:
###### Detail Kategori Program ######
# kategori_program  = {
#     # SEDEKAH UNTUK PALESTINA
#     'SEDEKAH KEMANUSIAAN: BANGUN KEMBALI PALESTINA' : 'SEDEKAH UNTUK PALESTINA', 
#     'BANGUN KEMBALI PALESTINA' : 'SEDEKAH UNTUK PALESTINA', 
#     'SEDEKAH PALESTINA' : 'SEDEKAH PALESTINA', 
#     'SEDEKAH UNTUK PALESTINA' : 'SEDEKAH UNTUK PALESTINA', 
#     'BANGUN KEMBALI GAZA' : 'SEDEKAH UNTUK PALESTINA', 
#     'INFAK PALESTINA' : 'SEDEKAH UNTUK PALESTINA', 
#     'INFAQ PALESTINA' : 'SEDEKAH UNTUK PALESTINA', 
#     'SEDEKAH SUBUH UNTUK PALESTINA' : 'SEDEKAH UNTUK PALESTINA', 
#     'SEDEKAH SUBUH UNTUK PALESTINA' : 'SEDEKAH UNTUK PALESTINA',
#     'INFAK UNTUK PALESTINA' : 'SEDEKAH UNTUK PALESTINA',  
#     'SEDEKAH SUBUH UNTUK PENGHAFAL QURAN GAZA' : 'SEDEKAH UNTUK PALESTINA',   
#     'HADIAH UNTUK GAZA PALESTINA' : 'SEDEKAH UNTUK PALESTINA', 
#     'SELAMATKAN PALESTINA DARI KELAPARAN!!' : 'SEDEKAH UNTUK PALESTINA', 
#     'PALESTINA' : 'SEDEKAH UNTUK PALESTINA',

#     'DOMBA PALESTINA' : 'SEDEKAH DAGING UNTUK PALESTINA',
#     'SEDEKAH DAGING PALESTINA' : 'SEDEKAH DAGING UNTUK PALESTINA',
    
#     'BERBAGI BUKA PUASA DENGAN YATIM PALESTINA' : 'RAMADHAN PALESTINA',
#     'BERBAGI BERBUKA YATIM PALESTINA' : 'RAMADHAN PALESTINA', 
#     'SEDEKAH KADO LEBARAN PALESTINA' : 'RAMADHAN PALESTINA', 
#     'KADO LEBARAN PALESTINA' : 'RAMADHAN PALESTINA', 
#     'SEDEKAH AIR BERSIH UNTUK PALESTINA' : 'RAMADHAN PALESTINA', 

#     'AL QURAN UNTUK PALESTINA' : 'AL QURAN UNTUK PALESTINA',

#     'ZAKAT PALESTINA' : 'ZAKAT UNTUK PALESTINA', 
#     'ZAKAT UNTUK PALESTINA' : 'ZAKAT UNTUK PALESTINA',

#     # SEDEKAH UMUM
#     'SEDEKAH' : 'SEDEKAH UMUM',
#     'INFAK' : 'SEDEKAH UMUM',
#     'SEDEKAH/INFAK HARIAN' : 'SEDEKAH UMUM', 
#     'SEDEKAH UMUM' : 'SEDEKAH UMUM',
#     'INFAK UMUM' : 'SEDEKAH UMUM',
#     'SEDEKAH SUBUH' : 'SEDEKAH UMUM', 
#     'SEDEKAH SHUBUH' : 'SEDEKAH UMUM', 
#     'SEDEKAH JUMAT' : 'SEDEKAH UMUM', 
#     'SEDEKAH JUMAT' : 'SEDEKAH UMUM', 
#     'SEDEKAH HARIAN' : 'SEDEKAH UMUM', 

#     # SEDEKAH JARIYAH
#     'SEDEKAH AIR BERSIH' : 'SEDEKAH JARIYAH', 
#     'PEMBEBASAN LAHAN SEKOLAH PENGHAFAL AL-QURAN' : 'SEDEKAH JARIYAH',  
#     'SEDEKAH JARIYAH SUMUR BOR' : 'SEDEKAH JARIYAH', 
#     'BANTU RENOVASI TEMPAT IBADAH DI PELOSOK DESA' : 'SEDEKAH JARIYAH',
#     'WAKAF MASJID' : 'SEDEKAH JARIYAH',
#     'WAKAF UMUM': 'SEDEKAH JARIYAH',
#     'SEDEKAH JARIYAH' : 'SEDEKAH JARIYAH',

#     # SEASONAL
#     'RAMADHAN' : 'SEASONAL',
#     'ZAKAT FITRAH' : 'SEASONAL',   
#     'SEDEKAH BUKA PUASA' : 'SEASONAL', 
#     'SEDEKAH KADO LEBARAN' : 'SEASONAL', 

#     'QURBAN FLASHSALE' : 'QURBAN',
#     'QURBAN' : 'QURBAN',

#     # SEDEKAH QURAN
#     'AL QURAN' : 'SEDEKAH AL QURAN',
#     'TEBAR MANFAAT QURAN' : 'SEDEKAH AL QURAN', 
#     'SEDEKAH QURAN, HADIAH TERINDAH DI AKHIRAT' : 'SEDEKAH AL QURAN',
#     'INFAK PROGRAM QURAN, AMAL TIADA HENTI' : 'SEDEKAH AL QURAN', 
#     'SEDEKAH PROGRAM QURAN, AMAL YANG TAK TERPUTUS' : 'SEDEKAH AL QURAN', 
#     'SEDEKAH SUBUH SANTRI PENGHAFAL QURAN' : 'SEDEKAH AL QURAN', 
#     'SEDEKAH AL QURAN' : 'SEDEKAH AL QURAN', 
#     'SEDEKAH QURAN' : 'SEDEKAH AL QURAN', 
#     'SEDEKAH QURAN UNTUK SANTRI' : 'SEDEKAH AL QURAN', 

#     # SEDEKAH UNTUK ANAK YATIM
#     'YATIM PIATU' : 'SEDEKAH UNTUK YATIM DAN DHUAFA',
#     'INFAK YATIM' : 'SEDEKAH UNTUK YATIM DAN DHUAFA', 
#     'BUKA PUASA YATIM DAN DUAFA' : 'SEDEKAH UNTUK YATIM DAN DHUAFA', 
#     'SEDEKAH YATIM DAN DHUAFA' : 'SEDEKAH UNTUK YATIM DAN DHUAFA', 
#     'SEDEKAH YATIM DAN  DHUAFA' : 'SEDEKAH UNTUK YATIM DAN DHUAFA', 
#     'SEDEKAH YATIM DHUAFA' : 'SEDEKAH UNTUK YATIM DAN DHUAFA', 
#     'INFAK YATIM DAN DUAFA' : 'SEDEKAH UNTUK YATIM DAN DHUAFA',
#     'SEDEKAH UNTUK ANAK YATIM' : 'SEDEKAH UNTUK YATIM DAN DHUAFA',
#     'SEDEKAH ANAK YATIM' : 'SEDEKAH UNTUK YATIM DAN DHUAFA',


#     # SEDEKAH UNTUK PENGOBATAN
#     'BANTU WILDAN SEMBUH DARI TUMOR STADIUM 3' : 'SEDEKAH UNTUK PENGOBATAN',
#     '1 TAHUN BERJUANG MELAWAN TUMOR STADIUM 3' : 'SEDEKAH UNTUK PENGOBATAN',
 
#     'FIDYAH' : 'FIDYAH', 
#     'LUNASI HUTANG PUASA DENGAN BAYAR FIDYAH' : 'FIDYAH', 

#     'SEDEKAH SUBUH UNTUK ORANG TUA' : 'SEDEKAH UNTUK ORANG TUA',
#     'SEDEKAH UNTUK ORANG TUA' : 'SEDEKAH UNTUK ORANG TUA',

#     'ZAKAT PENGHASILAN KITA UNTUK KEBERMANFAATAN' : 'ZAKAT PENGHASILAN/PROFESI',
#     'ZAKAT PENGHASILAN/PROFESI' : 'ZAKAT PENGHASILAN/PROFESI', 
#     'ZAKAT PENGHASILAN' : 'ZAKAT PENGHASILAN/PROFESI',

#     'LAYANAN AMBULANCE GRATIS' : 'LAYANAN AMBULANCE GRATIS', 
#     'INFAQ OPERASIONAL AMIL' : 'INFAQ OPERASIONAL AMIL', 
#     'DONASI BUNGA BANK' : 'DONASI BUNGA BANK',
#     'SEDEKAH BUKA PUASA SUNNAH' : 'SEDEKAH BUKA PUASA SUNNAH', 
#     'SEDEKAH BERAS RAKYAT' : 'SEDEKAH BERAS RAKYAT', 
#     'ZAKAT' : 'ZAKAT MAAL', 
#     'ZAKAT MAAL' : 'ZAKAT MAAL', 
#     'BEASISWA BINTANG' : 'BEASISWA BINTANG'
#     }

# # Asumsikan kolom program bernama 'Program'
# database_all['Kategori Program'] = database_all['Program'].map(kategori_program)

In [ ]:
### Kategori Program Donasi
# Daftar kondisi
conditions_program = [
                database_all['Program'].str.contains("PALESTINA|GAZA", case=False, na=False),
                database_all['Program'].str.contains("SUDAN", case=False, na=False),
                database_all['Program'].str.contains("QURAN|AL-QURAN", case=False, na=False),
                database_all['Program'].str.contains("MASJID|RENOVASI|AIR BERSIH|PEMBEBASAN LAHAN|JARIYAH|WAKAF|SUMUR", case=False, na=False),
                database_all['Program'].str.contains("YATIM", case=False, na=False),
                database_all['Program'].str.contains("QURBAN", case=False, na=False),
                database_all['Program'].str.contains("BEASISWA", case=False, na=False),
                database_all['Program'].str.contains("OPERASIONAL AMIL", case=False, na=False),
                database_all['Program'].str.contains("BUNGA BANK", case=False, na=False),
                database_all['Program'].str.contains("FIDYAH", case=False, na=False),
                database_all['Program'].str.contains("TUMOR", case=False, na=False),
                database_all['Program'].str.contains("AMBULANCE|AMBULANS", case=False, na=False),
                database_all['Program'].str.contains("HARIAN|SUBUH|SHUBUH|UMUM|SEDEKAH|INFAK", case=False, na=False),
                database_all['Program'].str.contains("KIFARAT|KAFARAT", case=False, na=False),
                database_all['Program'].str.contains("ZAKAT", case=False, na=False)
]


# Nilai untuk setiap kondisi
choices_program = [
    "PALESTINA",
    "SUDAN",
    "AL QURAN",
    "JARIYAH",
    "YATIM DAN DHUAFA",
    "QURBAN",
    "BEASISWA",
    "OPERASIONAL AMIL",
    "BUNGA BANK",
    "FIDYAH",
    "PENGOBATAN",
    "AMBULANCE GRATIS",
    "UMUM",
    "KAFARAT",
    "ZAKAT"
]

# 1. Buat Payment berdasar conditions & choices
database_all['Kategori Program'] = np.select(conditions_program, choices_program, default="LAINNYA")

In [60]:
lainnya_df = database_all[database_all['Kategori Program'] == 'LAINNYA']

# Lihat daftar program unik yang masuk kategori LAINNYA
program_lainnya = lainnya_df['Program'].unique()

print(program_lainnya)


['BERBAGI BUKA PUASA ARAFAH' 'GURU TELADAN'
 'BANGUN SUMUR BOR: PAHALA YANG TERUS MENGALIR' 'BANTU SUDAN SEKARANG'
 'PEDULI SUDAN : BANTU PERJUANGKAN KEMANUSIAAN DI SUDAN'
 'INFAQ KEMANUSIAAN SUDAN' "QUR'AN" 'NAN' 'AMBULANS GRATIS']


In [61]:
### Cek apakah ada kategori program yang kosong
kosong_kategori = database_all[database_all['Kategori Program'].isna()]
kosong_kategori['Program'].dropna().unique()

array([], dtype=object)

##### Standarisasi Payment Account

In [62]:
### Mengambil Data Payment Account jika Donatur menggunakan IPAYMU atau Taman Zakat Indonesia
## jika mengandung "Taman Zakat Indonesia"
mask_tzi = database_all['Payment Account'].str.fullmatch("TAMAN ZAKAT INDONESIA", case=False, na=False)
database_all.loc[mask_tzi, 'Payment Account'] = database_all.loc[mask_tzi, 'Payment Number']
## jika mengandung "IPAYMU"
mask_ipaymu = database_all['Payment Account'].str.fullmatch("IPAYMU", case=False, na=False)
database_all.loc[mask_ipaymu, 'Payment Account'] = database_all.loc[mask_ipaymu, 'Payment Number']

In [63]:
### Detail Metode Pembayaran ###
# Daftar kondisi
conditions_payment = [
                database_all['Payment Account'].str.contains("MANDIRI", case=False, na=False),
                database_all['Payment Account'].str.contains("MUAMALAT", case=False, na=False),
                database_all['Payment Account'].str.contains("PERMATA", case=False, na=False),
                database_all['Payment Account'].str.contains("CIMB|CIMB NIAGA", case=False, na=False),
                database_all['Payment Account'].str.contains("DANAMON", case=False, na=False),
                database_all['Payment Account'].str.contains("JATIM", case=False, na=False),
                database_all['Payment Account'].str.contains("BRI|021101002263302", case=False, na=False),
                database_all['Payment Account'].str.contains("BSI", case=False, na=False),
                database_all['Payment Account'].str.contains("BNI", case=False, na=False),
                database_all['Payment Account'].str.contains("BCA|2719095555", case=False, na=False),
                database_all['Payment Account'].str.contains("SEA BANK", case=False, na=False),
                database_all['Payment Account'].str.contains("SHOPEEPAY", case=False, na=False),
                database_all['Payment Account'].str.contains("DANA", case=False, na=False),
                database_all['Payment Account'].str.contains("OVO", case=False, na=False),
                database_all['Payment Account'].str.contains("LINKAJA", case=False, na=False),
                database_all['Payment Account'].str.contains("LINKITA", case=False, na=False),
                database_all['Payment Account'].str.contains("GOPAY", case=False, na=False),
                database_all['Payment Account'].str.contains("QRIS", case=False, na=False),
                database_all['Payment Account'].str.contains("BAG", case=False, na=False),
                database_all['Payment Account'].str.contains("BMI", case=False, na=False),
                database_all['Payment Account'].str.contains("PLASAMALL", case=False, na=False)
]

# Nilai untuk setiap kondisi
choices_payment = [
    "MANDIRI",
    "MUAMALAT",
    "PERMATA",
    "CIMB NIAGA",
    "DANAMON",
    "JATIM",
    "BRI",
    "BSI",
    "BNI",
    "BCA",
    "SEA BANK",
    "SHOPEEPAY",
    "DANA",
    "OVO",
    "LINKAJA",
    "LINKITA",
    "GOPAY",
    "QRIS",
    "BAG",
    "BMI",
    "PLASAMALL"
]

# 1. Buat Payment berdasar conditions & choices
database_all['Payment'] = np.select(conditions_payment, choices_payment, default="LAINNYA")

In [64]:
database_all[database_all['Payment'] == 'LAINNYA'].head()

,Donatur,Sapaan,Email,Date,Total,Comment,Payment Status,Payment Number,Payment Account,Fundraiser Name,Program,UTM Source,is_random,Whatsapp,Source,CRM,Catatan,Himpunan,Status,Rekap Himpunan,Unnamed: 7,Unnamed: 8,Nama Rekening,FU 1,FU 2,Report,Unnamed: 9,Unnamed: 10,Kolom2,Donasi,Kategori Program,Payment
120832,Andi Afandi,NaN,NaN,2025-10-17,150000,NaN,SUCCESS,NAN,NAN,NaN,QUR'AN DAN PALESTINA,NaN,True,6281236898516,NaN,WA 9009,ctwa e-book hr jum'at,WA 9009,Done,1HdG6IhI8pPDtTljfKTh7sgleGSdWpVrVTZmJWSj29Yc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PALESTINA,LAINNYA
120833,Yan Ari Nugraha,NaN,NaN,2025-10-04,50000,NaN,SUCCESS,NAN,NAN,NaN,SEDEKAH UMUM,NaN,True,628561670440,NaN,WA 5009,website aksi,WA 5009,Done,1HdG6IhI8pPDtTljfKTh7sgleGSdWpVrVTZmJWSj29Yc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UMUM,LAINNYA
120834,Rieski Diana Karmi,NaN,NaN,2025-10-13,50000,NaN,SUCCESS,NAN,NAN,NaN,SEDEKAH QURAN,NaN,True,6288803335075,NaN,WA 5009,tiba-tiba tf,WA 5009,Done,1HdG6IhI8pPDtTljfKTh7sgleGSdWpVrVTZmJWSj29Yc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL QURAN,LAINNYA
120835,Ade Rini,NaN,NaN,2025-10-18,100000,NaN,SUCCESS,NAN,NAN,NaN,SEDEKAH PALESTINA,NaN,True,6282116084312,NaN,WA 5009,tiba-tiba tf,WA 5009,Done,1HdG6IhI8pPDtTljfKTh7sgleGSdWpVrVTZmJWSj29Yc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PALESTINA,LAINNYA
121371,Widya,NaN,NaN,2025-06-27,100710,NaN,SUCCESS,NAN,NAN,NaN,SEDEKAH PALESTINA,NaN,True,6285795007433,NaN,WA 9009,NaN,NaN,NaN,1d7Bx_QmCoLPhWxi_3Gc1y0NA5Pf8hdY3SWwzWdbto-s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PALESTINA,LAINNYA


In [65]:
database_all['Whatsapp'].isnull().sum()

np.int64(0)

In [66]:
kosong_payment = database_all[database_all['Payment'] == 'LAINNYA']
kosong_payment['Payment Account'].dropna().unique()

array(['NAN', 'MIDTRANS', 'BJB', 'BRKSYARIAH', 'OCTO MOBILE', 'POSPAY',
       '-', '/', 'DB', '2025-09-14 00:00:00'], dtype=object)

In [67]:
# Tampilkan baris dengan Kategori Program yang kosong
database_all[database_all['Payment Account'].isna()]

,Donatur,Sapaan,Email,Date,Total,Comment,Payment Status,Payment Number,Payment Account,Fundraiser Name,Program,UTM Source,is_random,Whatsapp,Source,CRM,Catatan,Himpunan,Status,Rekap Himpunan,Unnamed: 7,Unnamed: 8,Nama Rekening,FU 1,FU 2,Report,Unnamed: 9,Unnamed: 10,Kolom2,Donasi,Kategori Program,Payment


In [68]:
# database_all.loc[database_all['Payment Status']=='NAN', 'Source']


In [69]:
# # Tampilkan baris dengan Kategori Program yang NAN
# database_all[(database_all['Payment Status']== 'NAN')]

In [70]:
### HAPUS DUPLIKAT SESAMA WEB ADS
# 1. Tandai baris duplikat berdasarkan tiga kolom
duplikat_mask_ads = database_all.duplicated(subset=['Date', 'Whatsapp', 'Total'], keep=False)
duplikat_rows_ads = database_all[duplikat_mask_ads].copy()

# 2. Buat fungsi filter untuk setiap grup duplikat
def resolve_duplicates(grup):
    unique_status = grup['Payment Status'].unique()
    
    if set(unique_status) == {'SUCCESS'} or set(unique_status) == {'WAITING'}:
        # Jika semua status sama, simpan satu saja
        return grup.head(1)
    elif 'SUCCESS' in unique_status and 'WAITING' in unique_status:
        # Jika ada campuran, buang WAITING
        return grup[grup['Payment Status'] != 'WAITING']
    else:
        # Default fallback (harusnya tidak terjadi)
        return grup.head(1)

# 3. Terapkan fungsi pada grup duplikat
filtered_duplikat_ads = duplikat_rows_ads.groupby(['Date', 'Whatsapp', 'Total'], group_keys=False).apply(resolve_duplicates)

# 4. Gabungkan kembali dengan data non-duplikat
non_duplikat_rows_ads = database_all[~duplikat_mask_ads]
database_all_cleaned_ads = pd.concat([non_duplikat_rows_ads, filtered_duplikat_ads], ignore_index=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_4764\3100286961.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_duplikat_ads = duplikat_rows_ads.groupby(['Date', 'Whatsapp', 'Total'], group_keys=False).apply(resolve_duplicates)


In [71]:
# Mengubah tipe data kolom 'Date' menjadi datetime
database_all_cleaned_ads['Date'] = pd.to_datetime(database_all_cleaned_ads['Date'], errors='coerce')

# Mengubah format Datetime menjadi 'dd/mm/yyyy'
database_all_cleaned_ads['Date'] = database_all_cleaned_ads['Date'].dt.strftime('%d/%m/%Y')

In [72]:
### Mengecek Donatur yang melakukan Retensi di WEB ADS

# Cek apakah ada baris duplikat berdasarkan tiga kolom
duplikat = database_all_cleaned_ads.duplicated(subset=['Date', 'Whatsapp', 'Total'], keep=False)

# Tampilkan semua baris yang terdeteksi sebagai duplikat
database_all_cleaned_ads[duplikat].to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/duplikat.xlsx', index=False)

In [73]:
#### HAPUS DUPLIKAT WEB ADS DAN RETENSI/CTWA
# 1. Tandai baris duplikat berdasarkan tiga kolom
duplikat_mask = database_all_cleaned_ads.duplicated(subset=['Date', 'Whatsapp', 'Total'], keep=False)
duplikat_rows = database_all_cleaned_ads[duplikat_mask].copy()

# 2. Buat fungsi filter untuk setiap grup duplikat
def resolve_duplicates(grup):
    unique_status = grup['Payment Status'].unique()
    
    if set(unique_status) == {'SUCCESS'} or set(unique_status) == {'WAITING'}:
        # Kalau semua status sama, prioritaskan yang Source = 'Web Ads'
        if 'Web Ads' in grup['Source'].values:
            return grup[grup['Source'] == 'Web Ads'].head(1)
        else:
            return grup.head(1)
    elif 'SUCCESS' in unique_status and 'WAITING' in unique_status:
        # Kalau ada campuran SUCCESS dan WAITING, buang WAITING
        grup = grup[grup['Payment Status'] != 'WAITING']
        # Setelah buang WAITING, tetap prioritaskan Web Ads jika ada
        if 'Web Ads' in grup['Source'].values:
            return grup[grup['Source'] == 'Web Ads'].head(1)
        else:
            return grup.head(1)
    else:
        # Fallback
        return grup.head(1)

# 3. Terapkan fungsi pada grup duplikat
filtered_duplikat = duplikat_rows.groupby(['Date', 'Whatsapp', 'Total'], group_keys=False).apply(resolve_duplicates)

# 4. Gabungkan kembali dengan data non-duplikat
non_duplikat_rows = database_all_cleaned_ads[~duplikat_mask]
database_all_cleaned = pd.concat([non_duplikat_rows, filtered_duplikat], ignore_index=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_4764\2346229834.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_duplikat = duplikat_rows.groupby(['Date', 'Whatsapp', 'Total'], group_keys=False).apply(resolve_duplicates)


In [74]:
database_all_cleaned['Date'] = pd.to_datetime(database_all_cleaned['Date'], errors='coerce')

C:\Users\HP\AppData\Local\Temp\ipykernel_4764\1920437727.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  database_all_cleaned['Date'] = pd.to_datetime(database_all_cleaned['Date'], errors='coerce')


In [75]:
database_all = database_all_cleaned

In [76]:
database_all['Payment Status'].unique()

array(['WAITING', 'SUCCESS', 'NAN'], dtype=object)

In [77]:
database_all['is_random'].value_counts()

is_random
True     127520
False      1991
Name: count, dtype: int64

In [78]:
database_all.to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Raw Data Transaksi 2025.xlsx', index=False)

#### Rename CRM

In [79]:
# database_all.loc[database_all['CRM'] == 'WA 5009', 'CRM'] = 'Syifa 5009'
# database_all.loc[database_all['CRM'] == 'WA 8004', 'CRM'] = 'Hana 8004'
# database_all.loc[database_all['CRM'] == 'WA 9009', 'CRM'] = 'Anisa 9009'
# database_all.loc[database_all['CRM'] == 'WA 0808', 'CRM'] = 'Wapin 0808'
# database_all.loc[database_all['CRM'] == 'WA 2028', 'CRM'] = 'Lia 2028'
# database_all.loc[database_all['CRM'] == 'WA 2029', 'CRM'] = 'Nia 2029'
# database_all.loc[database_all['CRM'] == 'WA 2031', 'CRM'] = 'Mira 2031'
# database_all.loc[database_all['CRM'] == 'WA 2032', 'CRM'] = 'Rani 2032'
# database_all.loc[database_all['CRM'] == 'WA 2033', 'CRM'] = 'Dina 2033'
# database_all.loc[database_all['CRM'] == 'WA 2034', 'CRM'] = 'Tia 2034'
# database_all.loc[database_all['CRM'] == 'WA 2035', 'CRM'] = 'Anisa 2035'
# database_all.loc[database_all['CRM'] == 'WA 2039', 'CRM'] = 'Anisa 2039'
# database_all.loc[database_all['CRM'] == 'WA 2037', 'CRM'] = 'Anisa 2037'
# database_all.loc[database_all['CRM'] == 'WA 2038', 'CRM'] = 'Anisa 2038'
# database_all.loc[database_all['CRM'] == 'WA 2040', 'CRM'] = 'Anisa 2040'
# database_all.loc[database_all['CRM'] == 'WA 2041', 'CRM'] = 'Anisa 2041'
# database_all.loc[database_all['CRM'] == 'WA 2042', 'CRM'] = 'Anisa 2042'

In [80]:
# Lihat value_counts Payment Status untuk CRM WA 2028
database_all['is_random'].value_counts(dropna=False)

is_random
True     127520
False      1991
Name: count, dtype: int64

In [81]:
database_all['Payment Status'].value_counts(dropna=False)

Payment Status
SUCCESS    105914
WAITING     22780
NAN           817
Name: count, dtype: int64

In [82]:
database_all[database_all['Payment Status'] == 'SUCCESS']['is_random'].value_counts(dropna=False)

is_random
True     104491
False      1423
Name: count, dtype: int64

In [83]:
### Ekspor data berpola
database_all[database_all['is_random']==False].to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Data Berpola.xlsx', index=False)

In [84]:
### Cek Payment untuk status WAITING
database_all[(database_all['Payment Status'] == 'WAITING')].to_excel('C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Data Waiting.xlsx', index=False)

In [85]:
# Filter data hanya yang nomor random dan payment status 'Success'
database = database_all[(database_all['is_random'] == True)
                        & (database_all['Payment Status'] == 'SUCCESS')
                        ]

In [86]:
database.loc[database['Whatsapp'] == 6287863317624, 'Source']

66752     Web Ads
101665    Web Ads
127187       CTWA
Name: Source, dtype: object

In [87]:
database['Whatsapp'].nunique()

81868

In [88]:
database_da['CRM'].value_counts(dropna=False)

CRM
WA 9009         6267
WA 5009         5177
WA 2029         5150
WA 8004         5015
WA 2032         4322
new2            2981
new1            2971
WA 2040         1500
WA 2042         1490
WA 2035         1420
WA 2039         1420
WA 2037         1082
new 2039        1002
new 2037        1002
new 2035        1000
WA 2041          866
WA 9956          864
WA 2047          864
WA 2052          863
WA 2055          863
WA 2053          863
WA 9954          863
WA 9953          863
WA 9952          863
WA 9951          863
WA 2065          863
WA 2043          863
WA 2050          863
WA 2054          863
WA 2044          863
WA 2048          863
WA 2049          863
WA 2046          856
refill 2047      651
refill 2065      637
refill 2055      637
refill1 2052     637
refill 2050      637
refill 2049      637
refill 2048      637
refill 9953      637
refill 2044      637
refill 2043      637
refill 9956      637
refill 2041      637
refill 9954      637
new 2054         637
new 2053 

In [89]:
database['CRM'].value_counts(dropna=False)

CRM
WA 9009         10230
WA 5009          7678
WA 8004          6912
WA 2029          6164
WA 2032          6136
NaN              5827
new2             3511
new1             3396
WA 2039          1975
WA 2035          1958
WA 2042          1930
WA 2040          1906
WA 2037          1719
WA 2043          1338
new 2037         1333
WA 2053          1289
WA 2047          1286
WA 2041          1236
new 2039         1207
WA 2048          1164
WA 2049          1153
new 2035         1137
WA 2054          1129
WA 2044          1114
WA 2055          1107
WA 9951          1106
WA 9952          1100
WA 2050          1093
WA 2065          1091
WA 9953          1066
WA 9954          1058
WA 9956          1043
WA 2052          1042
WA 2046          1037
refill 2044       771
refill 2046       762
refill 2043       754
refill 2041       742
refill 2047       740
refill 9954       723
refill 9951       721
refill 2048       716
new 2053          709
new 2054          709
refill 2050       707
refill

### TRACKING

#### Data Belum Pernah Sukses

In [90]:
# ##### TRACKING UNTUK DATA WAITING #####
# # 1. Pastikan kolom Date sebagai datetime
# database['Date'] = pd.to_datetime(database['Date'], errors='coerce')

# # Tambahkan kolom hari
# database['Hari'] = database['Date'].dt.day_name(locale='id_ID')

# # Tambahkan kolom masa gajian
# database['Tanggal'] = database['Date'].dt.day
# database['masa gajian'] = database['Tanggal'].apply(lambda x: 'Masa Gajian' if (x >= 26 or x <= 6) else 'Bukan Gajian')

# # 2. Buat kolom Bulan_Tahun
# database['Bulan_Tahun'] = database['Date'].dt.strftime('%b-%Y')

# # 3. Ambil data pertama per WA & bulan
# firsts = database.sort_values(by='Date').groupby(['Whatsapp', 'Bulan_Tahun'], as_index=False).first()

# # Ubah kolom Total ke numerik, jika ada string akan dikonversi jadi NaN
# database['Total'] = pd.to_numeric(database['Total'], errors='coerce')

# # 4. Total donasi per WA & bulan
# totals = database.groupby(['Whatsapp', 'Bulan_Tahun'], as_index=False)['Total'].sum()
# totals.rename(columns={'Total': 'Donasi'}, inplace=True)

# # Modus Hari
# modus_hari = database.groupby('Whatsapp')['Hari'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Tidak Diketahui').reset_index()
# modus_hari.rename(columns={'Hari': 'Modus Hari'}, inplace=True)

# # Modus Masa Gajian
# modus_gajian = database.groupby('Whatsapp')['masa gajian'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Tidak Diketahui').reset_index()
# modus_gajian.rename(columns={'masa gajian': 'Modus Masa Gajian'}, inplace=True)


# # 5. Gabung data pertama & total
# merged = pd.merge(firsts, totals, on=['Whatsapp', 'Bulan_Tahun'])

# # 6. Kolom yang diperlukan
# merged = merged[[ 
#     'Donatur', 'Whatsapp', 'Email', 'Bulan_Tahun',
#     'Kategori Program', 'Donasi', 'Date', 'Payment'
# ]]
# merged.columns = [
#     'Nama', 'WA', 'Email', 'Periode',
#     'Kategori Program', 'Donasi', 'Tanggal', 'Payment'
# ]

# # Pastikan hanya 1 baris per WA-Periode (buang duplikat WA di bulan yang sama)
# merged = merged.drop_duplicates(subset=['WA', 'Periode'])


# # 7. Pivot agar 1 baris per WA
# pivoted = merged.pivot_table(
#     index='WA',
#     columns='Periode',
#     values=['Kategori Program', 'Donasi', 'Tanggal', 'Payment'],
#     aggfunc=lambda x: x.iloc[0] if len(x) > 0 else None
# ).reset_index()


# # Rapikan nama kolom
# pivoted.columns = ['WA'] + [f"{col[0]} {col[1]}" for col in pivoted.columns[1:]]
# pivoted = pivoted.reset_index()

# # Ambil Nama & Email pertama per WA
# identitas = database.sort_values('Date').groupby('Whatsapp', as_index=False).first()
# identitas = identitas[['Whatsapp', 'Donatur', 'Email']]
# identitas.rename(columns={'Whatsapp': 'WA', 'Donatur': 'Nama'}, inplace=True)

# # Gabungkan ke pivot
# pivoted = pivoted.merge(identitas, on='WA', how='left')

# # Gabungkan source pertama WA
# first_source = database.sort_values('Date').groupby('Whatsapp', as_index=False).first()
# first_source = first_source[['Whatsapp', 'Source']]
# pivoted = pivoted.merge(first_source, left_on='WA', right_on='Whatsapp', how='left')
# pivoted.drop(columns='Whatsapp', inplace=True)


# # Urutkan kolom
# pivoted = pivoted[['Nama', 'WA', 'Email'] + [col for col in pivoted.columns if col not in ['Nama', 'WA', 'Email']]]

# # ========== TAMBAHAN ==========
# # 8. Frekuensi transaksi per WA
# # Hitung jumlah bulan unik per WA
# frekuensi = database.drop_duplicates(subset=['Whatsapp', 'Bulan_Tahun']) \
#     .groupby('Whatsapp').size().reset_index(name='frekuensi')
# frekuensi['kategori'] = frekuensi['frekuensi'].apply(
#     lambda x: 'Pasif' if x < 1 else 'Aktif' if x <= 4 else 'Loyal'
# )

# # 9. Total Donasi per WA
# total_donasi = database.groupby('Whatsapp')['Total'].sum().reset_index()
# total_donasi.rename(columns={'Total': 'Total Donasi'}, inplace=True)
# total_donasi['badge'] = total_donasi['Total Donasi'].apply(
#     lambda x: 'Bronze' if x < 100_000 else 'Silver' if x < 1_000_000 else 'Gold' if x < 5_000_000 else 'Platinum'
# )

# # 9.1 Preferensi: jumlah program baru yang belum pernah muncul sebelumnya per bulan per WA

# # Ambil set program per bulan
# program_per_bulan = database.groupby(['Whatsapp', 'Bulan_Tahun'])['Kategori Program'] \
#     .apply(lambda x: set(x)).reset_index()

# # Urutkan
# program_per_bulan = program_per_bulan.sort_values(['Whatsapp', 'Bulan_Tahun'])

# # Fungsi untuk menghitung program baru bulan demi bulan
# def hitung_preferensi(grup):
#     sudah_ada = set()
#     total_baru = 0
#     for prog_set in grup['Kategori Program']:
#         baru = prog_set - sudah_ada
#         total_baru += len(baru)
#         sudah_ada |= prog_set
#     return total_baru

# # Hitung preferensi per WA
# preferensi = program_per_bulan.groupby('Whatsapp').apply(hitung_preferensi).reset_index(name='Preferensi')


# # 9.2 Modus Program: program yang paling sering didonasikan
# modus_program = database.groupby('Whatsapp')['Kategori Program'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Tidak Diketahui').reset_index()
# modus_program.rename(columns={'Kategori Program': 'Program'}, inplace=True)

# # Modus dari Program (bukan Kategori Program)
# modus_program_real = database.groupby('Whatsapp')['Program'].agg(
#     lambda x: x.mode().iloc[0] if not x.mode().empty else 'Tidak Diketahui'
# ).reset_index()
# modus_program_real.rename(columns={'Program': 'Modus Program'}, inplace=True)

# # 10. Bulan & Tahun First Transaksi
# first_trx = database.sort_values('Date').groupby('Whatsapp', as_index=False).first()
# first_trx['Bulan'] = first_trx['Date'].dt.month
# first_trx['Tahun'] = first_trx['Date'].dt.year

# # 10. CRM Data
# crm_data = database.sort_values('Date').groupby('Whatsapp', as_index=False).first()
# crm_data = crm_data[['Whatsapp', 'CRM']]

# # Ambil tanggal donasi pertama dan sapaan pertama
# first_info = database.sort_values('Date').groupby('Whatsapp', as_index=False).first()

# # Kolom yang akan digunakan
# first_info = first_info[['Whatsapp', 'Date', 'Sapaan']]
# first_info.rename(columns={
#     'Whatsapp': 'WA',
#     'Date': 'Tanggal Donasi Pertama',
#     'Sapaan': 'Sapaan Pertama'
# }, inplace=True)

# # Gabungkan ke pivoted
# pivoted = pivoted.merge(first_info, on='WA', how='left')

# # 11. Gabungkan ke pivoted
# pivoted = pivoted.merge(frekuensi, left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')
# pivoted = pivoted.merge(total_donasi, left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')
# pivoted = pivoted.merge(first_trx[['Whatsapp', 'Bulan', 'Tahun']], left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')
# pivoted = pivoted.merge(preferensi, left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')
# pivoted = pivoted.merge(modus_program, left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')
# pivoted = pivoted.merge(modus_hari, left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')
# pivoted = pivoted.merge(modus_gajian, left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')
# pivoted = pivoted.merge(crm_data, left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')
# pivoted = pivoted.merge(modus_program_real, left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')


# # 12. Hapus kolom duplikat WA
# pivoted.drop(columns=[col for col in pivoted.columns if col.startswith('Whatsapp')], inplace=True)

# # ========== TAMBAHAN TRACKING PAYMENT ==========
# # 13. Tandai apakah WA sudah pernah success
# pernah_success = database.groupby('Whatsapp')['Payment Status'].apply(lambda x: 'SUCCESS' in x.values).reset_index()
# pernah_success.rename(columns={'Payment Status': 'Pernah Success'}, inplace=True)

# # Gabungkan ke pivoted
# pivoted = pivoted.merge(pernah_success, left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')

# # 14. Perbaiki kolom payment: jika belum pernah success dan nilainya 'waiting', ubah jadi 'Tetap Waiting'
# payment_cols = [col for col in pivoted.columns if col.startswith('Payment ')]

# for col in payment_cols:
#     pivoted[col] = pivoted.apply(
#         lambda row: row[col] if row['Pernah Success'] else 'Tetap Waiting' if row[col] == 'WAITING' else row[col],
#         axis=1
#     )

# # (Opsional) Tambahkan ringkasan status umum
# pivoted['Status Umum'] = pivoted['Pernah Success'].map({True: 'Sudah Pernah Success', False: 'Belum Pernah Success'})

# # Hapus kolom bantu jika tidak dibutuhkan
# pivoted.drop(columns=['Pernah Success'], inplace=True)

# # Buat list kolom awal, tapi hanya yang benar-benar ada di pivoted
# cols_awal_all = ['Nama', 'WA', 'Email', 'Bulan', 'Tahun', 'CRM', 'Source',
#                  'frekuensi', 'Preferensi', 'Program', 'Modus Program', 'kategori',
#                  'Total Donasi', 'badge', 'Modus Hari', 'Modus Masa Gajian']

# # Ambil hanya kolom yang ada di pivoted
# cols_awal = [col for col in cols_awal_all if col in pivoted.columns]

# # Gabungkan dengan kolom lainnya
# pivoted = pivoted[cols_awal + [col for col in pivoted.columns if col not in cols_awal]]


# ## ✅ Output
# # print(pivoted.head())


#### Data Sukses

In [91]:
database.info()

<class 'pandas.core.frame.DataFrame'>
Index: 104491 entries, 2 to 129510
Data columns (total 32 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Donatur           104485 non-null  object        
 1   Sapaan            39514 non-null   object        
 2   Email             14030 non-null   object        
 3   Date              104462 non-null  datetime64[ns]
 4   Total             104491 non-null  int64         
 5   Comment           24015 non-null   object        
 6   Payment Status    104491 non-null  object        
 7   Payment Number    104491 non-null  object        
 8   Payment Account   104491 non-null  object        
 9   Fundraiser Name   98009 non-null   object        
 10  Program           104491 non-null  object        
 11  UTM Source        13584 non-null   object        
 12  is_random         104491 non-null  bool          
 13  Whatsapp          104491 non-null  Int64         
 14  Source   

In [92]:
##### TRACKING UNTUK DATA SUCCESS #####
# 1. Pastikan kolom Date sebagai datetime
database['Date'] = pd.to_datetime(database['Date'], errors='coerce')

# Tambahkan kolom hari
database['Hari'] = database['Date'].dt.day_name(locale='id_ID')

# Tambahkan kolom masa gajian
database['Tanggal'] = database['Date'].dt.day
database['masa gajian'] = database['Tanggal'].apply(lambda x: 'Masa Gajian' if (x >= 26 or x <= 6) else 'Bukan Gajian')

# 2. Buat kolom Bulan_Tahun
database['Bulan_Tahun'] = database['Date'].dt.strftime('%b-%Y')

# 3. Ambil data pertama per WA & bulan
firsts = database.sort_values(by='Date').groupby(['Whatsapp', 'Bulan_Tahun'], as_index=False).first()

# Ubah kolom Total ke numerik, jika ada string akan dikonversi jadi NaN
database['Total'] = pd.to_numeric(database['Total'], errors='coerce')

# 4. Total donasi per WA & bulan
totals = database.groupby(['Whatsapp', 'Bulan_Tahun'], as_index=False)['Total'].sum()
totals.rename(columns={'Total': 'Donasi'}, inplace=True)

# Modus Hari
modus_hari = database.groupby('Whatsapp')['Hari'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Tidak Diketahui').reset_index()
modus_hari.rename(columns={'Hari': 'Modus Hari'}, inplace=True)

# Modus Masa Gajian
modus_gajian = database.groupby('Whatsapp')['masa gajian'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Tidak Diketahui').reset_index()
modus_gajian.rename(columns={'masa gajian': 'Modus Masa Gajian'}, inplace=True)


# 5. Gabung data pertama & total
merged = pd.merge(firsts, totals, on=['Whatsapp', 'Bulan_Tahun'])

# 6. Kolom yang diperlukan
merged = merged[[ 
    'Donatur', 'Whatsapp', 'Email', 'Bulan_Tahun',
    'Kategori Program', 'Donasi', 'Date', 'Payment'
]]
merged.columns = [
    'Nama', 'WA', 'Email', 'Periode',
    'Kategori Program', 'Donasi', 'Tanggal', 'Payment'
]

# Pastikan hanya 1 baris per WA-Periode (buang duplikat WA di bulan yang sama)
merged = merged.drop_duplicates(subset=['WA', 'Periode'])


# 7. Pivot agar 1 baris per WA
pivoted = merged.pivot_table(
    index='WA',
    columns='Periode',
    values=['Kategori Program', 'Donasi', 'Tanggal', 'Payment'],
    aggfunc=lambda x: x.iloc[0] if len(x) > 0 else None
).reset_index()


# Rapikan nama kolom
pivoted.columns = ['WA'] + [f"{col[0]} {col[1]}" for col in pivoted.columns[1:]]
pivoted = pivoted.reset_index()

# Ambil Nama & Email pertama per WA
identitas = database.sort_values('Date').groupby('Whatsapp', as_index=False).first()
identitas = identitas[['Whatsapp', 'Donatur', 'Email']]
identitas.rename(columns={'Whatsapp': 'WA', 'Donatur': 'Nama'}, inplace=True)

# Gabungkan ke pivot
pivoted = pivoted.merge(identitas, on='WA', how='left')

# Gabungkan source pertama WA
first_source = database.sort_values('Date').groupby('Whatsapp', as_index=False).first()
first_source = first_source[['Whatsapp', 'Source']]
pivoted = pivoted.merge(first_source, left_on='WA', right_on='Whatsapp', how='left')
pivoted.drop(columns='Whatsapp', inplace=True)


# Urutkan kolom
pivoted = pivoted[['Nama', 'WA', 'Email'] + [col for col in pivoted.columns if col not in ['Nama', 'WA', 'Email']]]


# ========== TAMBAHAN ==========
# 8. Frekuensi transaksi per WA
# Hitung jumlah bulan unik per WA
frekuensi = database.drop_duplicates(subset=['Whatsapp', 'Bulan_Tahun']) \
    .groupby('Whatsapp').size().reset_index(name='frekuensi')
frekuensi['kategori'] = frekuensi['frekuensi'].apply(
    lambda x: 'Pasif' if x < 1 else 'Aktif' if x <= 4 else 'Loyal'
)

# 9. Total Donasi per WA
total_donasi = database.groupby('Whatsapp')['Total'].sum().reset_index()
total_donasi.rename(columns={'Total': 'Total Donasi'}, inplace=True)
total_donasi['badge'] = total_donasi['Total Donasi'].apply(
    lambda x: 'Bronze' if x < 100_000 else 'Silver' if x < 1_000_000 else 'Gold' if x < 5_000_000 else 'Platinum'
)

# Hitung jumlah transaksi per WA
jumlah_transaksi = database.groupby('Whatsapp').size().reset_index(name='Jumlah Transaksi')
# Gabungkan ke pivoted
pivoted = pivoted.merge(jumlah_transaksi, left_on='WA', right_on='Whatsapp', how='left') \
                 .drop(columns='Whatsapp')

# 9.1 Preferensi: jumlah program baru yang belum pernah muncul sebelumnya per bulan per WA

# Ambil set program per bulan
program_per_bulan = database.groupby(['Whatsapp', 'Bulan_Tahun'])['Kategori Program'] \
    .apply(lambda x: set(x)).reset_index()

# Urutkan
program_per_bulan = program_per_bulan.sort_values(['Whatsapp', 'Bulan_Tahun'])

# Fungsi untuk menghitung program baru bulan demi bulan
def hitung_preferensi(grup):
    sudah_ada = set()
    total_baru = 0
    for prog_set in grup['Kategori Program']:
        baru = prog_set - sudah_ada
        total_baru += len(baru)
        sudah_ada |= prog_set
    return total_baru

# Hitung preferensi per WA
preferensi = program_per_bulan.groupby('Whatsapp').apply(hitung_preferensi).reset_index(name='Preferensi')


# 9.2 Modus Program: program yang paling sering didonasikan
modus_program = database.groupby('Whatsapp')['Kategori Program'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Tidak Diketahui').reset_index()
modus_program.rename(columns={'Kategori Program': 'Program'}, inplace=True)

# Modus dari Program (bukan Kategori Program)
modus_program_real = database.groupby('Whatsapp')['Program'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else 'Tidak Diketahui'
).reset_index()
modus_program_real.rename(columns={'Program': 'Modus Program'}, inplace=True)

# 10. Bulan & Tahun First Transaksi
first_trx = database.sort_values('Date').groupby('Whatsapp', as_index=False).first()
first_trx['Bulan'] = first_trx['Date'].dt.month
first_trx['Tahun'] = first_trx['Date'].dt.year

# 10. CRM Data
crm_data = database.sort_values('Date').groupby('Whatsapp', as_index=False).first()
crm_data = crm_data[['Whatsapp', 'CRM']]

# Ambil tanggal donasi pertama dan sapaan pertama
first_info = database.sort_values('Date').groupby('Whatsapp', as_index=False).first()

# Kolom yang akan digunakan
first_info = first_info[['Whatsapp', 'Date', 'Sapaan']]
first_info.rename(columns={
    'Whatsapp': 'WA',
    'Date': 'Tanggal Donasi Pertama',
    'Sapaan': 'Sapaan Pertama'
}, inplace=True)

# Gabungkan ke pivoted
pivoted = pivoted.merge(first_info, on='WA', how='left')

# Ambil tanggal transaksi terakhir per WA
last_info = database.sort_values('Date').groupby('Whatsapp', as_index=False).last()

# Pilih kolom yang dibutuhkan
last_info = last_info[['Whatsapp', 'Date']]
last_info.rename(columns={
    'Whatsapp': 'WA',
    'Date': 'Tanggal Transaksi Terakhir'
}, inplace=True)

# Gabungkan ke pivoted
pivoted = pivoted.merge(last_info, on='WA', how='left')


# 11. Gabungkan ke pivoted
pivoted = pivoted.merge(frekuensi, left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')
pivoted = pivoted.merge(total_donasi, left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')
pivoted = pivoted.merge(first_trx[['Whatsapp', 'Bulan', 'Tahun']], left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')
pivoted = pivoted.merge(preferensi, left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')
pivoted = pivoted.merge(modus_program, left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')
pivoted = pivoted.merge(modus_hari, left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')
pivoted = pivoted.merge(modus_gajian, left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')
pivoted = pivoted.merge(crm_data, left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')
pivoted = pivoted.merge(modus_program_real, left_on='WA', right_on='Whatsapp', how='left').drop(columns='Whatsapp')


# 12. Hapus kolom duplikat WA
pivoted.drop(columns=[col for col in pivoted.columns if col.startswith('Whatsapp')], inplace=True)

# Buat list kolom awal, tapi hanya yang benar-benar ada di pivoted
cols_awal_all = ['Nama', 'WA', 'Email', 'Bulan', 'Tahun', 'CRM', 'Source', 'Jumlah Transaksi',
                 'frekuensi', 'Preferensi', 'Program', 'Modus Program', 'kategori',
                 'Total Donasi', 'badge', 'Modus Hari', 'Modus Masa Gajian', 'Tanggal Transaksi Terakhir']

# Ambil hanya kolom yang ada di pivoted
cols_awal = [col for col in cols_awal_all if col in pivoted.columns]

# Gabungkan dengan kolom lainnya
pivoted = pivoted[cols_awal + [col for col in pivoted.columns if col not in cols_awal]]


## ✅ Output
# print(pivoted.head())


C:\Users\HP\AppData\Local\Temp\ipykernel_4764\2543396559.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  database['Date'] = pd.to_datetime(database['Date'], errors='coerce')
C:\Users\HP\AppData\Local\Temp\ipykernel_4764\2543396559.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  database['Hari'] = database['Date'].dt.day_name(locale='id_ID')
C:\Users\HP\AppData\Local\Temp\ipykernel_4764\2543396559.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [93]:
pivoted['WA'].duplicated().sum()

np.int64(0)

In [94]:
kategori_update['Whatsapp'].duplicated().sum()

np.int64(0)

In [95]:
print("Awal:", len(database))
print("Firsts:", len(firsts))
print("Merged:", len(merged))
print("Pivoted:", len(pivoted))
print("Frekuensi:", len(frekuensi))
print("Setelah merge frekuensi:", len(pivoted))  # ulangi setelah tiap merge


Awal: 104491
Firsts: 95769
Merged: 95769
Pivoted: 81868
Frekuensi: 81868
Setelah merge frekuensi: 81868


In [96]:
wa_frekuensi = set(frekuensi['Whatsapp'])
wa_pivoted   = set(pivoted['WA'])

hilang_frek = wa_frekuensi - wa_pivoted

if hilang_frek:
    raise ValueError(f"❌ ERROR: Terdapat {len(hilang_frek)} nomor WA yang hilang: {hilang_frek}")

print("✔ Semua nomor cocok, lanjut proses...")


✔ Semua nomor cocok, lanjut proses...


In [97]:
hilang_df = frekuensi[frekuensi['Whatsapp'].isin(hilang_frek)]
print(hilang_df.head())

Empty DataFrame
Columns: [Whatsapp, frekuensi, kategori]
Index: []


In [98]:
database[database['Whatsapp'].isin(hilang_frek)].sort_values(by='Whatsapp')

,Donatur,Sapaan,Email,Date,Total,Comment,Payment Status,Payment Number,Payment Account,Fundraiser Name,Program,UTM Source,is_random,Whatsapp,Source,CRM,Catatan,Himpunan,Status,Rekap Himpunan,Unnamed: 7,Unnamed: 8,Nama Rekening,FU 1,FU 2,Report,Unnamed: 9,Unnamed: 10,Kolom2,Donasi,Kategori Program,Payment,Hari,Tanggal,masa gajian,Bulan_Tahun


In [99]:
database[database['Whatsapp'].isin(hilang_frek)][
    ['Whatsapp', 'Date', 'Bulan_Tahun', 'Total', 'Kategori Program', 'CRM', 'Payment Status', 'Source']
].sort_values(by='Whatsapp')


,Whatsapp,Date,Bulan_Tahun,Total,Kategori Program,CRM,Payment Status,Source


In [100]:
# Hitung rata-rata donasi
pivoted['Rata-rata'] = pivoted['Total Donasi'] / pivoted['frekuensi']
pivoted['Rata-rata'] = pivoted['Rata-rata'].fillna(0).round(0)  # bulatkan dan hindari NaN
pivoted = pivoted.sort_values(by='Bulan', ascending=True)

pivoted['Range Rata-rata Donasi'] = pivoted['Rata-rata'].apply(
    lambda x: '<100.000' if x < 100_000 else '100.000 - 1.000.000' if x < 1_000_000 else '1.000.000 - 5.000.000' if x < 5_000_000 else '>5.000.000'
)

In [101]:
### Ganti nama kolom WA menjadi Whatsapp
pivoted.rename(columns={'WA': 'Whatsapp'}, inplace=True)

### Mengurutkan kolom
pivoted = pivoted[['Nama', 'Sapaan Pertama', 'Whatsapp', 'Email', 'Tanggal Donasi Pertama', 'Bulan', 'Tahun', 'CRM', 'Source',
                   'Kategori Program Jan-2025', 'Donasi Jan-2025','Tanggal Jan-2025', 'Payment Jan-2025',
                   'Kategori Program Feb-2025', 'Donasi Feb-2025','Tanggal Feb-2025', 'Payment Feb-2025',
                   'Kategori Program Mar-2025', 'Donasi Mar-2025','Tanggal Mar-2025', 'Payment Mar-2025',
                   'Kategori Program Apr-2025', 'Donasi Apr-2025','Tanggal Apr-2025', 'Payment Apr-2025',
                   'Kategori Program May-2025', 'Donasi May-2025','Tanggal May-2025', 'Payment May-2025',
                   'Kategori Program Jun-2025', 'Donasi Jun-2025','Tanggal Jun-2025', 'Payment Jun-2025',
                   'Kategori Program Jul-2025', 'Donasi Jul-2025','Tanggal Jul-2025', 'Payment Jul-2025',
                   'Kategori Program Aug-2025', 'Donasi Aug-2025','Tanggal Aug-2025', 'Payment Aug-2025',
                   'Kategori Program Sep-2025', 'Donasi Sep-2025','Tanggal Sep-2025', 'Payment Sep-2025',
                   'Kategori Program Oct-2025', 'Donasi Oct-2025','Tanggal Oct-2025', 'Payment Oct-2025',
                   'Kategori Program Nov-2025', 'Donasi Nov-2025','Tanggal Nov-2025', 'Payment Nov-2025',
                #    'Kategori Program Des-2025', 'Donasi Des-2025','Tanggal Des-2025', 'Payment Des-2025',
                   'Total Donasi','Jumlah Transaksi', 'frekuensi', 'Preferensi', 'Program', 'Modus Program', 'Rata-rata', 'kategori', 'badge',
                   'Range Rata-rata Donasi', 'Modus Hari', 'Modus Masa Gajian', 'Tanggal Transaksi Terakhir'
               #  , 'Status Umum'
                   ]]


#### Update Kategori Donatur

In [102]:
kategori_update = kategori_update.drop_duplicates(subset='Whatsapp', keep='first')

In [103]:
# Pastikan kolom Whatsapp punya tipe sama
pivoted['Whatsapp'] = pivoted['Whatsapp'].astype(str).str.strip()
kategori_update['Whatsapp'] = kategori_update['Whatsapp'].astype(str).str.strip()

# Merge langsung berdasarkan Whatsapp, menambahkan kategori, Validasi, dan Kategori_Prioritas
pivoted = pivoted.merge(
    kategori_update[['Whatsapp', 'No CS', 'kategori', 'Validasi', 'Kategori_Prioritas']],
    on='Whatsapp',
    how='left'
)

In [104]:
pivoted['Whatsapp'].duplicated().sum()

np.int64(0)

In [105]:
pivoted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81868 entries, 0 to 81867
Data columns (total 70 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Nama                        81866 non-null  object        
 1   Sapaan Pertama              35186 non-null  object        
 2   Whatsapp                    81868 non-null  object        
 3   Email                       12870 non-null  object        
 4   Tanggal Donasi Pertama      81868 non-null  datetime64[ns]
 5   Bulan                       81868 non-null  int32         
 6   Tahun                       81868 non-null  int32         
 7   CRM                         76628 non-null  object        
 8   Source                      81497 non-null  object        
 9   Kategori Program Jan-2025   43 non-null     object        
 10  Donasi Jan-2025             43 non-null     float64       
 11  Tanggal Jan-2025            43 non-null     datetime64

In [106]:
pivoted['kategori_x'].value_counts(dropna=False)

kategori_x
Aktif    81572
Loyal      296
Name: count, dtype: int64

In [107]:
pivoted['kategori_y'].value_counts(dropna=False)

kategori_y
Aktif        41050
NaN          23474
Pasif        14028
Eliminasi     3316
Name: count, dtype: int64

In [108]:
pivoted['CRM'].value_counts(dropna=False)

CRM
WA 9009         6566
WA 5009         5438
None            5240
WA 2029         5180
WA 8004         5024
WA 2032         4400
new2            2981
new1            2971
WA 2042         1502
WA 2040         1499
WA 2039         1433
WA 2035         1411
WA 2037         1079
new 2039        1002
new 2037        1002
new 2035        1000
WA 2055          879
WA 2041          877
WA 9953          873
WA 9951          871
WA 9952          871
WA 9956          869
WA 2047          869
WA 2050          867
WA 2054          867
WA 2044          865
WA 2065          865
WA 9954          864
WA 2048          864
WA 2053          864
WA 2049          863
WA 2043          863
WA 2052          862
WA 2046          856
refill 2047      651
refill 2044      637
refill 2055      637
refill 9951      637
refill 2043      637
refill 2041      637
refill 2048      637
refill 2049      637
new 2054         637
new 2053         637
refill 9953      637
refill 9952      637
refill 9956      637
refill1 2

In [109]:
kategori_update['kategori'].value_counts(dropna=False)

kategori
Aktif        41230
Pasif        14048
Eliminasi     3333
NaN              2
Name: count, dtype: int64

In [110]:
# Buat kolom kategori akhir
pivoted['kategori'] = pivoted['kategori_x']

# Timpa dengan kategori_y jika kategori_y adalah Pasif, Karantina, atau Eliminasi (case-insensitive)
mask = pivoted['kategori_y'].astype(str).str.strip().str.lower().isin(['pasif', 'karantina', 'eliminasi'])
pivoted.loc[mask, 'kategori'] = pivoted.loc[mask, 'kategori_y']

# Hapus kolom sementara
pivoted.drop(columns=['kategori_x', 'kategori_y'], inplace=True, errors='ignore')

In [111]:
pivoted['kategori'].value_counts(dropna=False)

kategori
Aktif        64293
Pasif        14028
Eliminasi     3316
Loyal          231
Name: count, dtype: int64

In [112]:
#### Mengubah kategori berdasarkan tanggal transaksi terakhir
# Tentukan batas tanggal
batas = pd.to_datetime("2025-06-25")

# Pastikan kolom tanggal benar
pivoted["Tanggal Transaksi Terakhir"] = pd.to_datetime(
    pivoted["Tanggal Transaksi Terakhir"], errors="coerce"
)

# Buat kolom penanda perubahan (default False)
pivoted["Pernah_Diubah"] = False

# Mask kondisi perubahan
mask_kategori = (
    (pivoted["kategori"].str.lower() == "pasif") &
    (pivoted["Kategori_Prioritas"].str.lower() == "karantina") &
    (pivoted["Tanggal Transaksi Terakhir"] > batas)
)

# Simpan baris mana saja yang akan berubah
pivoted.loc[mask_kategori, "Pernah_Diubah"] = True

# Update kategori & kategori_prioritas
pivoted.loc[mask_kategori, "kategori"] = "aktif"
pivoted.loc[mask_kategori, "Kategori_Prioritas"] = "prioritas 1"

In [113]:
pivoted['CRM'].value_counts(dropna=False)

CRM
WA 9009         6566
WA 5009         5438
None            5240
WA 2029         5180
WA 8004         5024
WA 2032         4400
new2            2981
new1            2971
WA 2042         1502
WA 2040         1499
WA 2039         1433
WA 2035         1411
WA 2037         1079
new 2039        1002
new 2037        1002
new 2035        1000
WA 2055          879
WA 2041          877
WA 9953          873
WA 9951          871
WA 9952          871
WA 9956          869
WA 2047          869
WA 2050          867
WA 2054          867
WA 2044          865
WA 2065          865
WA 9954          864
WA 2048          864
WA 2053          864
WA 2049          863
WA 2043          863
WA 2052          862
WA 2046          856
refill 2047      651
refill 2044      637
refill 2055      637
refill 9951      637
refill 2043      637
refill 2041      637
refill 2048      637
refill 2049      637
new 2054         637
new 2053         637
refill 9953      637
refill 9952      637
refill 9956      637
refill1 2

In [114]:
pivoted['Kategori_Prioritas'].value_counts(dropna=False)  

Kategori_Prioritas
prioritas 3        31220
NaN                24803
karantina          12762
prioritas 2         6238
eliminasi           3532
prioritas 1         3204
tidak diketahui      109
Name: count, dtype: int64

In [115]:
pivoted['Validasi'].value_counts(dropna=False)

Validasi
valid      53535
NaN        23473
invalid     3531
Invalid     1329
Name: count, dtype: int64

In [116]:
# pivoted.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Tracking Taman Zakat 2025.xlsx", index=False)

### Database Final

In [117]:
database_final = pivoted[['Nama', 'Sapaan Pertama', 'Whatsapp', 'Tanggal Donasi Pertama', 'Bulan', 'Tahun', 'CRM', 'Source', 'Jumlah Transaksi',
                          'frekuensi', 'Preferensi', 'Program', 'Modus Program', 'Rata-rata', 'Total Donasi', 'kategori', 'badge',
                          'Range Rata-rata Donasi', 'Modus Hari', 'Modus Masa Gajian', 'Validasi', 'Kategori_Prioritas', 'Tanggal Transaksi Terakhir'
                        #   , 'Status Umum'
                          ]]

In [118]:
database_final['Whatsapp'] = pd.to_numeric(database_final['Whatsapp'], errors='coerce')

C:\Users\HP\AppData\Local\Temp\ipykernel_4764\2769925172.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  database_final['Whatsapp'] = pd.to_numeric(database_final['Whatsapp'], errors='coerce')


In [119]:
# database_final['Status Umum'].value_counts(dropna=False)

In [120]:
database_final.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/DATABASE.xlsx", index=False)

In [121]:
# #### Mengambil data donatur yang BELUM PERNAH SUKSES ####
# # Filter data yang Payment Status-nya 'WAITING'
# data_waiting = database_final[(database_final['Status Umum'] == 'Belum Pernah Success') &
#                         (database_final['Source'] == 'Web Ads')]

# # Hitung value_counts dari kolom CRM
# data_waiting['CRM'].value_counts(dropna=False)

# data_waiting.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/Database Belum Pernah Sukses.xlsx", index=False)

### Data Invalid & Pasif

In [122]:
### Menyimpan database donatur invalid ###
database_invalid = database_final[database_final['Validasi'].str.strip().str.lower() == 'invalid'][[
    'CRM', 'Bulan', 'Nama', 'Whatsapp', 'Program', 'Rata-rata', 'kategori',
    'badge', 'Modus Hari', 'Modus Masa Gajian',
    'Validasi', 'Kategori_Prioritas', 'Tanggal Transaksi Terakhir'
    # , 'Status Umum'
]]

# Simpan database_meta ke Excel
database_invalid.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/DATABASE INVALID.xlsx", index=False)

In [123]:
### Menyimpan database donatur pasif ###
database_pasif = database_final[database_final['kategori'].str.strip().str.lower() == 'pasif'][[
    'CRM', 'Bulan', 'Nama', 'Whatsapp', 'Program', 'Rata-rata', 'kategori',
    'badge', 'Modus Hari', 'Modus Masa Gajian',
    'Validasi', 'Kategori_Prioritas', 'Tanggal Transaksi Terakhir'
    # , 'Status Umum'
]]
# Simpan database_meta ke Excel
database_pasif.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/DATABASE PASIF.xlsx", index=False)

In [124]:
### Menyimpan database donatur pasif ###
database_eliminasi = database_final[database_final['kategori'].str.strip().str.lower() == 'eliminasi'][[
    'CRM', 'Bulan', 'Nama', 'Whatsapp', 'Program', 'Rata-rata', 'kategori',
    'badge', 'Modus Hari', 'Modus Masa Gajian',
    'Validasi', 'Kategori_Prioritas', 'Tanggal Transaksi Terakhir'
    # , 'Status Umum'
]]
# Simpan database_meta ke Excel
database_eliminasi.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/OUTPUT/DATABASE ELIMINASI.xlsx", index=False)

### Pemetaan CS

In [125]:
# database_final = pd.read_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/DATABASE.xlsx")

In [126]:
database_final['kategori'].value_counts(dropna=False)

kategori
Aktif        64293
Pasif        13013
Eliminasi     3316
aktif         1015
Loyal          231
Name: count, dtype: int64

In [127]:
database_meta = database_final[
    (
        (database_final['Validasi'].str.strip().str.lower() != 'invalid')
    ) 
    &
    (database_final['kategori'].str.strip().str.lower() != 'pasif') 
    # & (database_final['kategori'].str.strip().str.lower() != 'eliminasi')
][[
    'CRM', 'Bulan', 'Nama', 'Whatsapp', 'Program', 'Rata-rata', 'kategori',
    'Source', 'badge', 'Modus Hari', 'Modus Masa Gajian',
    'Validasi', 'Kategori_Prioritas', 'Tanggal Transaksi Terakhir'
    # , 'Status Umum'
]]


In [128]:
database_meta['Kategori_Prioritas'].value_counts(dropna=False)

Kategori_Prioritas
prioritas 3        31220
NaN                23473
prioritas 2         6238
prioritas 1         3204
tidak diketahui      109
Name: count, dtype: int64

In [129]:
### Menentukan Rekomendasi Hari untuk Prospek Donatur
mask_bukan_jumat = ~database_meta['Modus Hari'].str.fullmatch("Jumat", case=False, na=False)
database_meta.loc[mask_bukan_jumat, 'Modus Hari'] = database_meta.loc[mask_bukan_jumat, 'Modus Masa Gajian']

In [130]:
# Atur urutan kategori untuk Modus Masa Gajian
database_meta['Modus Hari'] = pd.Categorical(
    database_meta['Modus Hari'],
    categories=["Jumat", "Masa Gajian", "Bukan Gajian"],
    ordered=True
)

# Atur urutan kategori untuk badge
database_meta['badge'] = pd.Categorical(
    database_meta['badge'],
    categories=["Bronze", "Silver", "Gold", "Platinum"],
    ordered=True
)

# Urutkan berdasarkan: Modus Masa Gajian, Program (alfabet), Badge
database_meta_sorted = database_meta.sort_values(
    by=['Modus Hari', 'Program', 'badge']
)

In [131]:
#DROP data yang tidak diperlukan
database_meta_sorted = database_meta_sorted.drop(['Modus Masa Gajian'], axis=1)

# menyesuaikan nama kolom jika ada nama kolom yang belum sesuai standarisasi
kolom = {"Modus Hari": "Periode Blast"}
# rename
database_meta_sorted = database_meta_sorted.rename(columns=kolom)

In [132]:
database_meta_sorted['CRM'].value_counts(dropna=False)

CRM
WA 2032         4018
None            3866
WA 2029         3267
new2            2776
new1            2626
WA 9009         2318
WA 5009         2203
WA 2039         1411
WA 2035         1404
WA 2040         1359
WA 2042         1342
new 2039        1001
new 2035        1000
WA 8004          977
WA 2037          962
new 2037         951
WA 2052          860
WA 2046          856
WA 2055          810
WA 2054          803
WA 2044          802
WA 9953          801
WA 9951          798
WA 2053          797
WA 2043          796
WA 2065          794
WA 9956          786
WA 2050          786
WA 9952          784
WA 2047          782
WA 9954          781
WA 2041          781
WA 2049          781
WA 2048          778
refill 2047      651
refill 2065      637
refill 2050      637
refill 2044      637
refill1 2052     637
refill 9954      637
refill 9956      637
refill 9953      637
refill 9951      637
refill 2048      637
refill 2055      637
refill 9952      637
refill 2043      637
refill 20

In [133]:
database_meta_sorted.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/DATABASE CRM.xlsx", index=False)

In [134]:
database_meta_sorted['Whatsapp'].duplicated().sum()

np.int64(0)

In [164]:
WA_5009 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 5009']
WA_8004 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 8004']
WA_9009 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 9009']
WA_0808 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 0808']
WA_2028 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2028']
WA_2029 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2029']
WA_8131 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 8131']
WA_2032 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2032']
WA_2033 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2033']
WA_2035 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2035']
WA_2037 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2037']
WA_2038 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2038']
WA_2039 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2039']
WA_2040 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2040']
WA_2041 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2041']
WA_2042 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2042']
WA_2043 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2043']
WA_2044 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2044']
WA_2046 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2046']
WA_2047 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2047']
WA_2048 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2048']
WA_2049 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2049']
WA_2050 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2050']
WA_2052 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2052']
WA_2053 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2053']
WA_2054 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2054']
WA_2055 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2055']
WA_2065 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 2065']
WA_9951 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 9951']
WA_9952 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 9952']
WA_9953 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 9953']
WA_9954 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 9954']
WA_9956 = database_meta_sorted[database_meta_sorted['CRM'] == 'WA 9956']
WA_None = database_meta_sorted[database_meta_sorted['CRM'].isna()]
new1 = database_meta_sorted[database_meta_sorted['CRM'] == 'new1']
new2 = database_meta_sorted[database_meta_sorted['CRM'] == 'new2']
new2037 = database_meta_sorted[database_meta_sorted['CRM'] == 'new 2037']
new2053 = database_meta_sorted[database_meta_sorted['CRM'] == 'new 2053']
new2054 = database_meta_sorted[database_meta_sorted['CRM'] == 'new 2054']
refill_2040 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 2040']
refill_2041 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 2041']
refill_2042 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 2042']
refill_2043 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 2043']
refill_2044 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 2044']
refill_2046 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 2046']
refill_2047 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 2047']
refill_2048 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 2048']
refill_2049 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 2049']
refill_2050 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 2050']
refill_2052 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill1 2052']
refill_2053 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 2053']
refill_2054 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 2054']
refill_2055 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 2055']
refill_2065 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 2065']
refill_9951 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 9951']
refill_9952 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 9952']
refill_9953 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 9953']
refill_9954 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 9954']
refill_9956 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill 9956']
refill2_2037 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 2037']
refill2_2040 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 2040']
refill2_2041 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 2041']
refill2_2042 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 2042']
refill2_2043 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 2043']
refill2_2044 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 2044']
refill2_2046 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 2046']
refill2_2047 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 2047']
refill2_2048 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 2048']
refill2_2049 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 2049']
refill2_2050 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 2050']
refill2_2052 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 2052']
refill2_2053 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 2053']
refill2_2054 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 2054']
refill2_2055 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 2055']
refill2_2065 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 2065']
refill2_9951 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 9951']
refill2_9952 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 9952']
refill2_9953 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 9953']
refill2_9954 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 9954']
refill2_9956 = database_meta_sorted[database_meta_sorted['CRM'] == 'refill2 9956']


# # Simpan masing-masing DataFrame ke file Excel terpisah
# WA_5009.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_5009.xlsx", index=False)
# WA_8004.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/Widya 8004.xlsx", index=False)
# WA_9009.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA 9009.xlsx", index=False)
# WA_0808.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/Wapin 0808.xlsx", index=False)
# WA_2028.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2028.xlsx", index=False)
# WA_2029.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA 2029.xlsx", index=False)
# WA_8131.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA 8131.xlsx", index=False)
# WA_2032.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA 2032.xlsx", index=False)
# WA_2033.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2033.xlsx", index=False)
# WA_2035.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/Samsul 2035.xlsx", index=False)
# WA_2037.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2037.xlsx", index=False)
# WA_2038.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/Nadiya 2038.xlsx", index=False)
# WA_2039.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/Dian 2039.xlsx", index=False)
# WA_2040.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2040.xlsx", index=False)
# WA_2041.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2041.xlsx", index=False)
# WA_2042.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2042.xlsx", index=False)
# WA_2043.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2043.xlsx", index=False)
# WA_2044.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2044.xlsx", index=False)
# WA_2046.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2046.xlsx", index=False)
# WA_2047.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2047.xlsx", index=False)
# WA_2048.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2048.xlsx", index=False)
# WA_2049.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2049.xlsx", index=False)
# WA_2050.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2050.xlsx", index=False)
# WA_2052.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2052.xlsx", index=False)
# WA_2053.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2053.xlsx", index=False)
# WA_2054.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2054.xlsx", index=False)
# WA_2055.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2055.xlsx", index=False)
# WA_2065.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_2065.xlsx", index=False)
# WA_9951.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_9951.xlsx", index=False)
# WA_9952.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_9952.xlsx", index=False)
# WA_9953.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_9953.xlsx", index=False)
# WA_9954.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_9954.xlsx", index=False)
# WA_9956.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_9956.xlsx", index=False)
WA_None.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_None.xlsx", index=False)
# new1.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/suntikan_udin.xlsx", index=False)
# new2.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/Widya_New2.xlsx", index=False)
# new2037.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_New2037.xlsx", index=False)
# new2053.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_New2053.xlsx", index=False)
# new2054.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_New2054.xlsx", index=False)
# refill_2040.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2040.xlsx", index=False)
# refill_2041.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2041.xlsx", index=False)
# refill_2042.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2042.xlsx", index=False)
# refill_2043.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2043.xlsx", index=False)
# refill_2044.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2044.xlsx", index=False)
# refill_2046.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2046.xlsx", index=False)
# refill_2047.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2047.xlsx", index=False)
# refill_2048.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2048.xlsx", index=False)
# refill_2049.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2049.xlsx", index=False)
# refill_2050.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2050.xlsx", index=False)
refill_2052.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2052.xlsx", index=False)
# refill_2053.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2053.xlsx", index=False)
# refill_2054.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2054.xlsx", index=False)
# refill_2055.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2055.xlsx", index=False)
# refill_2065.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2065.xlsx", index=False)
# refill_9951.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill9951.xlsx", index=False)
# refill_9952.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill9952.xlsx", index=False)
# refill_9953.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill9953.xlsx", index=False)
# refill_9954.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill9954.xlsx", index=False)
# refill_9956.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill9956.xlsx", index=False)

refill2_2037.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_2037.xlsx", index=False)
# refill2_2040.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_2040.xlsx", index=False)
# refill2_2041.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_2041.xlsx", index=False)
# refill2_2042.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_2042.xlsx", index=False)
refill2_2043.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_2043.xlsx", index=False)
# refill2_2044.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_2044.xlsx", index=False)
# refill2_2046.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_2046.xlsx", index=False)
# refill2_2047.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_2047.xlsx", index=False)
# refill2_2048.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_2048.xlsx", index=False)
# refill2_2049.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_2049.xlsx", index=False)
# refill2_2050.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_2050.xlsx", index=False)
# refill2_2052.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_2052.xlsx", index=False)
# refill2_2053.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_2053.xlsx", index=False)
# refill2_2054.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_2054.xlsx", index=False)
# refill2_2055.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_2055.xlsx", index=False)
# refill2_2065.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_2065.xlsx", index=False)
# refill2_9951.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_9951.xlsx", index=False)
# refill2_9952.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_9952.xlsx", index=False)
# refill2_9953.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_9953.xlsx", index=False)
# refill2_9954.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_9954.xlsx", index=False)
# refill2_9956.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/Data CRM/WA_Refill2_9956.xlsx", index=False)

In [136]:
# irisan9009

In [137]:
# data = pd.read_excel(
#     'C:/Users/HP/OneDrive/Documents/KERJAAA/QURBAN.xlsx', 
#     sheet_name = 'Keken')             # Jumlah baris data
# data.head()

In [138]:
# qurban = data.merge(database_final, left_on='Whatsapp', right_on='WA', how='left').drop(columns='Whatsapp')

In [139]:
# qurban.info()

In [140]:
# qurban.to_excel("C:/Users/HP/OneDrive/Documents/KERJAAA/qurban keken.xlsx", index=False)

### Cek TOP 10

In [141]:
###  Total Donatur per bulan dan Donatur Baru per bulan
# 1. Format tanggal dan buat kolom bulan
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data['Bulan'] = data['Date'].dt.to_period('M')  # Contoh: 2025-07

# 2. Filter transaksi sukses
success_data = data[data['Payment Status'].str.lower().str.strip() == 'success']

# 3. Ambil bulan pertama transaksi sukses dari masing-masing donatur
donatur_pertama = success_data.groupby('Whatsapp')['Bulan'].min().reset_index()
donatur_pertama.columns = ['Whatsapp', 'Bulan_Pertama']

# 4. Hitung donatur baru per bulan
donatur_baru_per_bulan = donatur_pertama.groupby('Bulan_Pertama')['Whatsapp'].nunique().reset_index()
donatur_baru_per_bulan.columns = ['Bulan', 'Donatur_Baru']

# 5. Hitung total donatur aktif per bulan
total_donatur_per_bulan = success_data.groupby('Bulan')['Whatsapp'].nunique().reset_index()
total_donatur_per_bulan.columns = ['Bulan', 'Total_Donatur']

# 6. Gabungkan keduanya
summary_new_donatur = pd.merge(donatur_baru_per_bulan, total_donatur_per_bulan, on='Bulan', how='outer').sort_values('Bulan')

## Print hasil
print("Jumlah Donatur Baru per Bulan:")
print(summary_new_donatur)

Jumlah Donatur Baru per Bulan:
      Bulan  Donatur_Baru  Total_Donatur
0   2025-01            43             43
1   2025-02           379            387
2   2025-03         15610          15661
3   2025-04          9934          10607
4   2025-05         10236          11372
5   2025-06         11469          13089
6   2025-07          7651           8983
7   2025-08          5839           6913
8   2025-09          7338           8535
9   2025-10          8845          10121
10  2025-11          6038           6245


In [142]:
## Akumulasi Donatur Unik per Bulan berdasarkan CRM
# Pastikan kolom tanggal
database_retensi['Tanggal'] = pd.to_datetime(database_retensi['Tanggal'], errors='coerce')
database_retensi['Bulan_Tahun'] = database_retensi['Tanggal'].dt.to_period('M').astype(str)

# Hitung donatur unik per bulan per CRM
donatur_per_bulan = (
    database_retensi.groupby(['Bulan_Tahun', 'CRM'])['Whatsapp']
    .nunique()
    .reset_index(name='Donatur_Unik')
)

# Hitung akumulasi donatur per CRM
donatur_per_bulan = donatur_per_bulan.sort_values(['CRM', 'Bulan_Tahun'])
donatur_per_bulan['Donatur_Akumulasi'] = (
    donatur_per_bulan.groupby('CRM')['Donatur_Unik'].cumsum()
)

# Transpose: CRM jadi baris, Bulan_Tahun jadi kolom
tabel_transposed = donatur_per_bulan.pivot_table(
    index='CRM',
    columns='Bulan_Tahun',
    values='Donatur_Akumulasi',
    fill_value=0  # Jika ingin kosong jadi 0
)

# Optional: Reset index jika ingin jadikan CRM kolom
# tabel_transposed = tabel_transposed.reset_index()

# Mengubah tipe data kolom menjadi integer
tabel_transposed = tabel_transposed.fillna(0).astype(int)

# Tampilkan hasil akhir
print(tabel_transposed)

Bulan_Tahun  2025-03  2025-04  2025-05  2025-06  2025-07  2025-08  2025-09  \
CRM                                                                          
WA 2028            0        0        0        0        0       69      127   
WA 2032            0        0        0       57      201      286      352   
WA 2035            0        0        0        0       37       76      121   
WA 2037            0        0        0        2       16       43       85   
WA 2038            0        0        0        1       14        0        0   
WA 2039            0        0        0        7       24       75      103   
WA 2040            0        0        0        0        1       21       50   
WA 2041            0        0        0        0        0       10       33   
WA 2042            0        0        0        0        2       23       55   
WA 2043            0        0        0        0        0        2       16   
WA 2044            0        0        0        0        0        

In [143]:
database_retensi['CRM'].value_counts(dropna=False)

CRM
WA 9009    1440
WA 5009    1072
WA 2032     677
WA 8004     482
WA 2028     313
WA 2037     224
WA 2035     203
WA 2043     191
WA 2042     172
WA 2039     166
WA 2041     165
WA 2040     161
WA 2048     135
WA 2047     130
WA 2049     125
WA 2053     125
WA 2054     121
WA 2050     116
WA 2055     116
WA 9951     111
WA 2044     103
WA 9952      98
WA 2065      98
WA 9956      87
WA 9954      69
WA 9953      65
WA 2038      14
WA 2052       8
Name: count, dtype: int64

In [144]:
# Pastikan kolom tanggal dan donasi valid
database_retensi['Tanggal'] = pd.to_datetime(database_retensi['Tanggal'], errors='coerce')
database_retensi['Bulan_Tahun'] = database_retensi['Tanggal'].dt.to_period('M').astype(str)

# Hitung total donasi per bulan per CRM
donasi_per_bulan = (
    database_retensi.groupby(['Bulan_Tahun', 'CRM'])['Donasi']
    .sum()
    .reset_index(name='Total_Donasi')
)

# Transpose: CRM jadi baris, Bulan_Tahun jadi kolom
tabel_donasi_transposed = donasi_per_bulan.pivot_table(
    index='CRM',
    columns='Bulan_Tahun',
    values='Total_Donasi',
    fill_value=0
)

# Ubah tipe ke integer jika ingin (atau pakai float jika nominal besar dengan desimal)
tabel_donasi_transposed = tabel_donasi_transposed.astype(int)

# Tampilkan hasil akhir
print(tabel_donasi_transposed)


Bulan_Tahun   2025-03   2025-04   2025-05   2025-06   2025-07   2025-08  \
CRM                                                                       
WA 2028             0         0         0         0         0   7212335   
WA 2032             0         0         0   7200761  20762088  12743163   
WA 2035             0         0         0         0   5000000   6405500   
WA 2037             0         0         0    300000   2550000   4775025   
WA 2038             0         0         0    200000   2053698         0   
WA 2039             0         0         0   1195000   3090000   9947275   
WA 2040             0         0         0         0     60000   3550000   
WA 2041             0         0         0         0         0    730500   
WA 2042             0         0         0         0    150000   1945000   
WA 2043             0         0         0         0         0    150081   
WA 2044             0         0         0         0         0    100000   
WA 2047             0    

In [145]:
#### Rekap Donatur Sukses per Hari
# Pastikan kolom Date dalam format datetime
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Filter rentang tanggal
start_date = '2025-06-01'
end_date = '2025-06-30'

# Filter data sukses di rentang tanggal
filtered = data[
    (data['Date'] >= start_date) &
    (data['Date'] <= end_date) &
    (data['Payment Status'].str.lower().str.strip() == 'success')
]

# Ambil nama hari dalam Bahasa Inggris
filtered['Nama Hari'] = filtered['Date'].dt.day_name()

# Opsional: mapping ke Bahasa Indonesia
hari_mapping = {
    'Monday': 'Senin',
    'Tuesday': 'Selasa',
    'Wednesday': 'Rabu',
    'Thursday': 'Kamis',
    'Friday': 'Jumat',
    'Saturday': 'Sabtu',
    'Sunday': 'Minggu'
}
filtered['Nama Hari'] = filtered['Nama Hari'].map(hari_mapping)

# Hitung jumlah donatur unik per nama hari
jumlah_per_hari = filtered.groupby('Nama Hari')['Whatsapp'].nunique().reindex([
    'Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat', 'Sabtu', 'Minggu'
]).fillna(0).reset_index()

jumlah_per_hari.columns = ['Hari', 'Jumlah Donatur Sukses']
print(jumlah_per_hari)


     Hari  Jumlah Donatur Sukses
0   Senin                   1645
1  Selasa                   1932
2    Rabu                   1932
3   Kamis                   2124
4   Jumat                   2493
5   Sabtu                   1424
6  Minggu                   1918


C:\Users\HP\AppData\Local\Temp\ipykernel_4764\3045077799.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Nama Hari'] = filtered['Date'].dt.day_name()
C:\Users\HP\AppData\Local\Temp\ipykernel_4764\3045077799.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Nama Hari'] = filtered['Nama Hari'].map(hari_mapping)


In [146]:
#### Jumlah Retensi Per Hari
# Pastikan kolom Date dalam format datetime
database_retensi['Tanggal'] = pd.to_datetime(database_retensi['Tanggal'], errors='coerce')

# Filter rentang tanggal
start_date = '2025-06-01'
end_date = '2025-06-30'

# Filter data sukses di rentang tanggal
filtered = database_retensi[
    (database_retensi['Tanggal'] >= start_date) &
    (database_retensi['Tanggal'] <= end_date) & 
    (database_retensi['Payment Status'].str.lower().str.strip() == 'success')
]

# Ambil nama hari dalam Bahasa Inggris
filtered['Nama Hari'] = filtered['Tanggal'].dt.day_name()

# Opsional: mapping ke Bahasa Indonesia
hari_mapping = {
    'Monday': 'Senin',
    'Tuesday': 'Selasa',
    'Wednesday': 'Rabu',
    'Thursday': 'Kamis',
    'Friday': 'Jumat',
    'Saturday': 'Sabtu',
    'Sunday': 'Minggu'
}
filtered['Nama Hari'] = filtered['Nama Hari'].map(hari_mapping)

# Hitung jumlah donatur unik per nama hari
jumlah_per_hari = filtered.groupby('Nama Hari')['Whatsapp'].nunique().reindex([
    'Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat', 'Sabtu', 'Minggu'
]).fillna(0).reset_index()

jumlah_per_hari.columns = ['Hari', 'Jumlah Donatur Sukses']
print(jumlah_per_hari)


     Hari  Jumlah Donatur Sukses
0   Senin                     61
1  Selasa                     82
2    Rabu                     69
3   Kamis                     93
4   Jumat                    153
5   Sabtu                     70
6  Minggu                     92


C:\Users\HP\AppData\Local\Temp\ipykernel_4764\614062079.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Nama Hari'] = filtered['Tanggal'].dt.day_name()
C:\Users\HP\AppData\Local\Temp\ipykernel_4764\614062079.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Nama Hari'] = filtered['Nama Hari'].map(hari_mapping)


In [147]:
### Menghitung banyak donatur yang melakukan retensi
start_date = '2025-01-01'
end_date = '2025-06-25'

filtered = database_retensi[
    (database_retensi['Tanggal'] >= start_date) &
    (database_retensi['Tanggal'] <= end_date)
]

filtered.groupby('CRM')['Whatsapp'].nunique().reset_index(name='Jumlah WA Unik')


,CRM,Jumlah WA Unik
0,WA 2032,23
1,WA 2039,6
2,WA 5009,211
3,WA 8004,148
4,WA 9009,394


In [148]:
# Hitung jumlah CTWA Whatsapp unik per CRM
print("jumlah CTWA Whatsapp unik per CRM")
ctwa.groupby('CRM')['Whatsapp'].nunique().reset_index(name='Jumlah WA Unik')

jumlah CTWA Whatsapp unik per CRM


,CRM,Jumlah WA Unik
0,WA 2028,176
1,WA 2029,174
2,WA 2032,159
3,WA 2035,123
4,WA 2039,30
5,WA 2055,206
6,WA 5009,956
7,WA 9009,193
8,WA 9951,206
9,WA 9952,218


In [149]:
# Filter hanya yang Payment Status-nya 'Success'
ctwa_success = ctwa[ctwa['Payment Status'] == 'Success']

# Hitung jumlah Whatsapp unik per CRM
ctwa_success_per_crm = ctwa_success.groupby('CRM')['Whatsapp'].nunique().reset_index(name='Jumlah WA Success')

print(ctwa_success_per_crm)


         CRM  Jumlah WA Success
0    WA 2029                 33
1    WA 2032                 11
2    WA 2035                  2
3    WA 2055                 10
4    WA 5009                282
5    WA 9009                 54
6    WA 9951                  8
7    WA 9952                  4
8    WA 9953                 10
9   new 2035                  1
10  new 2039                  1


In [150]:
print("Data Web Ads")

# Hitung Jumlah Whatsapp
jumlah_wa = len(data['Whatsapp'])
print(f"Jumlah Transaksi: {jumlah_wa}")

# Jumlah Whatsapp unik
jumlah_wa_unik = data['Whatsapp'].nunique()
print(f"Jumlah WA unik: {jumlah_wa_unik}")

# Jumlah Whatsapp unik yang pernah success
jumlah_wa_success = data[data['Payment Status'] == 'Success']['Whatsapp'].nunique()
print(f"Jumlah WA unik dengan Payment Status Success: {jumlah_wa_success}")

# Jumlah Whatsapp unik yang waiting
jumlah_wa_waiting = data[data['Payment Status'] == 'Waiting']['Whatsapp'].nunique()
print(f"Jumlah WA unik dengan Payment Status Waiting: {jumlah_wa_waiting}")

Data Web Ads
Jumlah Transaksi: 120824
Jumlah WA unik: 100076
Jumlah WA unik dengan Payment Status Success: 83382
Jumlah WA unik dengan Payment Status Waiting: 19617


In [151]:
### Waiting Success Per bulan
# Pastikan Date dalam datetime dan buat kolom Bulan_Tahun
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data['Bulan_Tahun'] = data['Date'].dt.to_period('M').astype(str)

# Hitung jumlah status per bulan
status_summary = data.groupby(['Bulan_Tahun', 'Payment Status']).size().reset_index(name='Jumlah')

# Transpose Payment Status jadi kolom
status_pivot = status_summary.pivot_table(index='Bulan_Tahun', columns='Payment Status', values='Jumlah', fill_value=0).reset_index()

# Mengubah tipe data kolom menjadi integer
status_pivot[['Success', 'Waiting']] = status_pivot[['Success', 'Waiting']].fillna(0).astype(int)

# Tampilkan hasil
print(status_pivot)


Payment Status Bulan_Tahun  Success  Waiting
0                  2025-01       52       22
1                  2025-02      417      127
2                  2025-03    16669     3705
3                  2025-04    11367     2429
4                  2025-05    12414     2793
5                  2025-06    14166     3247
6                  2025-07     9938     1487
7                  2025-08     7532     1188
8                  2025-09     9230     3106
9                  2025-10    10820     2206
10                 2025-11     6881     1028


In [152]:
data_ads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120824 entries, 0 to 120823
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Donatur          120823 non-null  object        
 1   Sapaan           49205 non-null   object        
 2   Email            17452 non-null   object        
 3   Date             120824 non-null  datetime64[ns]
 4   Total            120824 non-null  int64         
 5   Comment          29229 non-null   object        
 6   Payment Status   120824 non-null  object        
 7   Payment Number   120824 non-null  object        
 8   Payment Account  120824 non-null  object        
 9   Fundraiser Name  120824 non-null  object        
 10  Program          120824 non-null  object        
 11  UTM Source       17039 non-null   object        
 12  is_random        120824 non-null  bool          
 13  Whatsapp         120824 non-null  int64         
 14  Source           120

In [153]:
### Waiting Success Per bulan
# Pastikan Date dalam datetime dan buat kolom Bulan_Tahun
data_ads['Date'] = pd.to_datetime(data_ads['Date'], errors='coerce')
data_ads['Bulan_Tahun'] = data_ads['Date'].dt.to_period('M').astype(str)

# Hitung jumlah status per bulan
random_summary = data_ads.groupby(['Bulan_Tahun', 'is_random']).size().reset_index(name='Jumlah')

# Transpose Payment Status jadi kolom
random_pivot = random_summary.pivot_table(index='Bulan_Tahun', columns='is_random', values='Jumlah', fill_value=0).reset_index()
# Transpose Payment Status jadi kolom
random_pivot = random_summary.pivot_table(
    index='Bulan_Tahun', 
    columns='is_random', 
    values='Jumlah', 
    fill_value=0
).reset_index()

# Pastikan nama kolom boolean jadi string
random_pivot.columns = random_pivot.columns.map(lambda x: str(x))

# Ubah tipe data kolom jadi integer
if 'True' in random_pivot.columns and 'False' in random_pivot.columns:
    random_pivot[['True', 'False']] = random_pivot[['True', 'False']].astype(int)

# Tampilkan hasil
print(random_pivot)

is_random Bulan_Tahun  False   True
0             2025-01      2     72
1             2025-02     15    529
2             2025-03    302  20072
3             2025-04    281  13515
4             2025-05    327  14880
5             2025-06    339  17074
6             2025-07    257  11168
7             2025-08    209   8511
8             2025-09    201  12135
9             2025-10     58  12968
10            2025-11      0   7909


In [154]:
fig = px.bar(
    status_summary,
    x='Bulan_Tahun',
    y='Jumlah',
    color='Payment Status',
    barmode='relative',  # Stacked bar
    title='Jumlah SUCCESS dan WAITING per Bulan (Stacked) Web Ads',
    labels={'Bulan_Tahun': 'Bulan', 'Jumlah': 'Jumlah Transaksi'}
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [155]:
# Jumlah Whatsapp unik
jumlah_ctwa_wa_unik = ctwa['Whatsapp'].nunique()
print(f"Jumlah CTWA unik: {jumlah_ctwa_wa_unik}")

# Jumlah Whatsapp unik yang pernah success
jumlah_ctwa_wa_success = ctwa[ctwa['Payment Status'] == 'Success']['Whatsapp'].nunique()
print(f"Jumlah CTWA unik dengan Payment Status Success: {jumlah_ctwa_wa_success}")

# Jumlah Whatsapp unik yang waiting
jumlah_ctwa_wa_waiting = ctwa[ctwa['Payment Status'] == 'Waiting']['Whatsapp'].nunique()
print(f"Jumlah CTWA unik dengan Payment Status Waiting: {jumlah_ctwa_wa_waiting}")


Jumlah CTWA unik: 2591
Jumlah CTWA unik dengan Payment Status Success: 412
Jumlah CTWA unik dengan Payment Status Waiting: 1038


In [156]:
# Pastikan Date datetime
ctwa['Tanggal'] = pd.to_datetime(ctwa['Tanggal'], errors='coerce')
ctwa['Bulan_Tahun'] = ctwa['Tanggal'].dt.to_period('M').astype(str)
# Standarisasi kolom Payment Status
ctwa['Payment Status'] = ctwa['Payment Status'].str.strip().str.upper()


# Hitung jumlah status per bulan
status_summary_ctwa = ctwa.groupby(['Bulan_Tahun', 'Payment Status']).size().reset_index(name='Jumlah')

fig = px.bar(
    status_summary_ctwa,
    x='Bulan_Tahun',
    y='Jumlah',
    color='Payment Status',
    barmode='relative',  # Stacked bar
    title='Jumlah SUCCESS dan WAITING per Bulan (Stacked) Web Ads',
    labels={'Bulan_Tahun': 'Bulan', 'Jumlah': 'Jumlah Transaksi'}
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [157]:
### Mencari Data Donatur yang belum ada CRM atau belum terdistribusi
# Pastikan kolom 'Bulan' dalam bentuk integer (jika belum)
database_meta_sorted['Bulan'] = database_meta_sorted['Bulan'].astype(int)

# Filter data yang belum memiliki CRM (kosong atau NaN)
belum_ada_crm = database_meta_sorted[database_meta_sorted['CRM'].isna()]

# Hitung jumlah per bulan
belum_ada_crm_per_bulan = belum_ada_crm.groupby('Bulan').size().reset_index(name='Jumlah_Belum_CRM')

# Optional: Jika ingin urut berdasarkan bulan
belum_ada_crm_per_bulan = belum_ada_crm_per_bulan.sort_values('Bulan')

# Tampilkan hasil
print(belum_ada_crm_per_bulan)

   Bulan  Jumlah_Belum_CRM
0      5                38
1      6                16
2      7                 2
3     10                29
4     11              3781


In [158]:
### PIC Donatur
database_meta_sorted['CRM'].value_counts(dropna=False)

CRM
WA 2032         4018
None            3866
WA 2029         3267
new2            2776
new1            2626
WA 9009         2318
WA 5009         2203
WA 2039         1411
WA 2035         1404
WA 2040         1359
WA 2042         1342
new 2039        1001
new 2035        1000
WA 8004          977
WA 2037          962
new 2037         951
WA 2052          860
WA 2046          856
WA 2055          810
WA 2054          803
WA 2044          802
WA 9953          801
WA 9951          798
WA 2053          797
WA 2043          796
WA 2065          794
WA 9956          786
WA 2050          786
WA 9952          784
WA 2047          782
WA 9954          781
WA 2041          781
WA 2049          781
WA 2048          778
refill 2047      651
refill 2065      637
refill 2050      637
refill 2044      637
refill1 2052     637
refill 9954      637
refill 9956      637
refill 9953      637
refill 9951      637
refill 2048      637
refill 2055      637
refill 9952      637
refill 2043      637
refill 20

In [159]:
#### Jumlah Valid & Invalid Whatsapp Donatur (Tidak Berpola)
# Pastikan kolom tanggal sudah benar
database['Date'] = pd.to_datetime(database['Date'], errors='coerce')
database['Bulan_Tahun'] = database['Date'].dt.to_period('M').astype(str)

# Hitung jumlah is_random per bulan
is_random_summary = (
    database.groupby(['Bulan_Tahun', 'is_random'])
    .size()
    .reset_index(name='Jumlah')
)

# Tambahkan kolom akumulasi berdasarkan is_random
is_random_summary = is_random_summary.sort_values(['is_random', 'Bulan_Tahun'])
is_random_summary['Akumulasi'] = (
    is_random_summary.groupby('is_random')['Jumlah'].cumsum()
)

print(is_random_summary)

   Bulan_Tahun  is_random  Jumlah  Akumulasi
0      2025-01       True      52         52
1      2025-02       True     409        461
2      2025-03       True   16459      16920
3      2025-04       True   11403      28323
4      2025-05       True   12691      41014
5      2025-06       True   14504      55518
6      2025-07       True   10473      65991
7      2025-08       True    8323      74314
8      2025-09       True   10109      84423
9      2025-10       True   12115      96538
10     2025-11       True    7924     104462
11         NaT       True      29     104491


C:\Users\HP\AppData\Local\Temp\ipykernel_4764\2390918640.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp\ipykernel_4764\2390918640.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [160]:
database_final['Validasi'].value_counts(dropna=False)

Validasi
valid      53535
NaN        23473
invalid     3531
Invalid     1329
Name: count, dtype: int64

In [161]:
database_final['kategori'].value_counts(dropna=False)

kategori
Aktif        64293
Pasif        13013
Eliminasi     3316
aktif         1015
Loyal          231
Name: count, dtype: int64

In [162]:
# top_10_nominal = database.nlargest(10, 'Total')
# print(top_10_nominal)